# 🌦️ Создание вашей первой команды интеллектуальных агентов: Прогрессивный погодный бот с ADK

Этот туториал является расширенной версией [руководства по быстрому старту](https://google.github.io/adk-docs/get-started/quickstart/) для **Agent Development Kit (ADK)**. Теперь вы готовы погрузиться глубже и создать более сложную **многоагентную систему**.

Мы займемся созданием **команды погодных ботов**, постепенно добавляя продвинутые функции на простую основу. Начав с одного агента, который умеет узнавать погоду, мы последовательно добавим такие возможности, как:

*   Использование различных AI-моделей (Gemini, GPT, Claude).
*   Проектирование специализированных подагентов для отдельных задач (например, приветствия и прощания).
*   Включение интеллектуального делегирования между агентами.
*   Наделение агентов памятью с помощью постоянного состояния сессии.
*   Реализация критически важных защитных механизмов (guardrails) с использованием колбэков.

### 🤔 Почему именно команда погодных ботов?

Этот пример, хоть и кажется простым, представляет собой практическую и понятную основу для изучения ключевых концепций ADK, необходимых для создания сложных агентных приложений в реальном мире. Вы научитесь структурировать взаимодействия, управлять состоянием, обеспечивать безопасность и оркестрировать совместную работу нескольких AI-«мозгов».

### ⚙️ Что такое ADK?

Напомним, ADK — это Python-фреймворк, предназначенный для упрощения разработки приложений на базе больших языковых моделей (LLM). Он предлагает надежные строительные блоки для создания агентов, которые могут рассуждать, планировать, использовать инструменты, динамически взаимодействовать с пользователями и эффективно сотрудничать в команде.

### ✅ В этом продвинутом туториале вы освоите:

*   **Определение и использование инструментов:** Создание Python-функций (`tools`), которые наделяют агентов специфическими способностями (например, получение данных), и обучение агентов их эффективному использованию.
*   **Гибкость в выборе LLM:** Настройка агентов для использования различных ведущих LLM (Gemini, GPT-4o, Claude Sonnet) через интеграцию с LiteLLM, что позволяет выбирать лучшую модель для каждой задачи.
*   **Делегирование и коллаборация агентов:** Проектирование специализированных подагентов и включение автоматической маршрутизации (`auto flow`) запросов пользователя к наиболее подходящему агенту в команде.
*   **Состояние сессии для памяти:** Использование `Session State` и `ToolContext` для того, чтобы агенты могли запоминать информацию между ходами диалога, что ведет к более контекстуальным взаимодействиям.
*   **Защитные механизмы (Guardrails) с колбэками:** Реализация `before_model_callback` и `before_tool_callback` для проверки, изменения или блокировки запросов/использования инструментов на основе предопределенных правил, повышая безопасность и контроль над приложением.

### 🎯 Ожидаемый результат

По завершении этого туториала вы создадите функционирующую многоагентную систему погодных ботов. Эта система будет не только предоставлять информацию о погоде, но и обрабатывать вежливые обороты речи, запоминать последний проверенный город и работать в рамках определенных границ безопасности — и все это будет оркестрировано с помощью ADK.

### 📋 Предварительные требования:

*   ✅ **Уверенное знание языка программирования Python.**
*   ✅ **Знакомство с большими языковыми моделями (LLM), API и концепцией агентов.**
*   ❗ **Критически важно: Завершение руководства ADK Quickstart или наличие эквивалентных базовых знаний ADK (Agent, Runner, SessionService, основы использования Tool).** Этот туториал напрямую основывается на этих концепциях.
*   ✅ **API-ключи** для LLM, которые вы собираетесь использовать (например, из Google AI Studio для Gemini, OpenAI Platform, Anthropic Console).

---

### 📝 Примечание о среде выполнения

Этот туториал предназначен для интерактивных сред, таких как Google Colab, Colab Enterprise или Jupyter Notebooks. Пожалуйста, учтите следующее:

*   **Запуск асинхронного кода:** Среды ноутбуков обрабатывают асинхронный код по-разному. Вы увидите примеры с использованием `await` (подходит, когда цикл событий уже запущен, что характерно для ноутбуков) или `asyncio.run()` (часто требуется при запуске в виде отдельного `.py` скрипта или в специфических конфигурациях ноутбуков). Блоки кода содержат рекомендации для обоих сценариев.
*   **Ручная настройка Runner/Session:** Шаги включают явное создание экземпляров `Runner` и `SessionService`. Этот подход показан, потому что он дает вам детальный контроль над жизненным циклом выполнения агента, управлением сессиями и сохранением состояния.

### 💡 Альтернатива: Использование встроенных инструментов ADK (Web UI / CLI / API Server)

Если вы предпочитаете настройку, которая автоматически управляет `Runner` и сессиями с помощью стандартных инструментов ADK, вы можете найти эквивалентный код, структурированный для этой цели, [здесь](https://github.com/google/adk-docs/tree/main/examples/python/tutorial/agent_team/adk-tutorial). Эта версия предназначена для прямого запуска с помощью команд, таких как `adk web` (для веб-интерфейса), `adk run` (для взаимодействия через CLI) или `adk api_server` (для предоставления API). Пожалуйста, следуйте инструкциям в `README.md`, предоставленным в этом альтернативном ресурсе.

---

**Готовы создать свою команду агентов? Давайте начнем!**


In [1]:
# @title Шаг 0: Настройка и установка
# Установка ADK и LiteLLM для поддержки нескольких моделей

!pip install google-adk -q
!pip install litellm -q

print("Установка завершена.")

You should consider upgrading via the '/Users/me/Documents/TWRB/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/me/Documents/TWRB/venv/bin/python3 -m pip install --upgrade pip' command.
Установка завершена.


In [ ]:
# @title Импорт необходимых библиотек
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm          # Для поддержки нескольких моделей
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types                          # Для создания контента/частей сообщения

import warnings
# Игнорировать все предупреждения
warnings.filterwarnings("ignore")

import logging
# Настройка логирования для вывода только ошибок
logging.basicConfig(level=logging.ERROR)

print("Библиотеки импортированы.")

/Users/me/Documents/TWRB/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Библиотеки импортированы.


In [ ]:
# @title Настройка ключей API (Замените на ваши реальные ключи!)

# --- ВАЖНО: Замените заглушки на ваши реальные ключи API ---


# Ключ API Gemini (Получите в Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "ВАШ_GOOGLE_API_KEY" # <--- ЗАМЕНИТЬ

# [Необязательно]
# Ключ API OpenAI (Получите на платформе OpenAI: https://platform.openai.com/api-keys)
os.environ['OPENAI_API_KEY'] = 'ВАШ_OPENAI_API_KEY' # <--- ЗАМЕНИТЬ

# [Необязательно]
# Ключ API Anthropic (Получите в консоли Anthropic: https://console.anthropic.com/settings/keys)
os.environ['ANTHROPIC_API_KEY'] = 'ВАШ_ANTHROPIC_API_KEY' # <--- ЗАМЕНИТЬ


# --- Проверка ключей (необязательно) ---
print("Ключи API установлены:")
print(f"Ключ API Google установлен: {'Да' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'ВАШ_GOOGLE_API_KEY' else 'Нет (ЗАМЕНИТЕ ЗАГЛУШКУ!)'}")
print(f"Ключ API OpenAI установлен: {'Да' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'ВАШ_OPENAI_API_KEY' else 'Нет (ЗАМЕНИТЕ ЗАГЛУШКУ!)'}")
print(f"Ключ API Anthropic установлен: {'Да' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'ВАШ_ANTHROPIC_API_KEY' else 'Нет (ЗАМЕНИТЕ ЗАГЛУШКУ!)'}")

# Настройка ADK для прямого использования ключей API (без Vertex AI для этой мультимодельной конфигурации)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Примечание по безопасности:** Рекомендуется безопасно управлять ключами API (например, с помощью секретов Colab или переменных окружения), а не прописывать их напрямую в коде. Замените строки-заглушки выше.

Ключи API установлены:
Ключ API Google установлен: Да
Ключ API OpenAI установлен: Да
Ключ API Anthropic установлен: Нет (ЗАМЕНИТЕ ЗАГЛУШКУ!)


In [4]:
# --- Определение констант моделей для удобства использования ---

# Больше поддерживаемых моделей можно найти здесь: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# Больше поддерживаемых моделей можно найти здесь: https://docs.litellm.ai/docs/providers/openai#openai-chat-completion-models
MODEL_GPT_4O = "openai/gpt-4.1" # Вы также можете попробовать: gpt-4.1-mini, gpt-4o и т.д.

# Больше поддерживаемых моделей можно найти здесь: https://docs.litellm.ai/docs/providers/anthropic
MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-20250514" # Вы также можете попробовать: claude-opus-4-20250514, claude-3-7-sonnet-20250219 и т.д.

print("\nОкружение настроено.")


Окружение настроено.


---

## 🤖 Шаг 1: Ваш первый агент — базовый поиск погоды

Начнем с создания фундаментального компонента нашего погодного бота: одного агента, способного выполнять конкретную задачу — поиск информации о погоде. Это включает в себя создание двух ключевых частей:

1.  **Инструмент (Tool):** Python-функция, которая наделяет агента *способностью* получать данные о погоде.
2.  **Агент (Agent):** AI-«мозг», который понимает запрос пользователя, знает о наличии у него инструмента для погоды и решает, когда и как его использовать.

---

### 🛠️ 1. Определяем инструмент (`get_weather`)

В ADK **инструменты (Tools)** — это строительные блоки, которые дают агентам конкретные возможности, выходящие за рамки простой генерации текста. Обычно это обычные Python-функции, выполняющие определенные действия, такие как вызов API, запрос к базе данных или выполнение вычислений.

Наш первый инструмент будет предоставлять *имитацию* прогноза погоды. Это позволяет нам сосредоточиться на структуре агента, пока не требуя внешних API-ключей. Позже вы сможете легко заменить эту имитационную функцию на ту, что вызывает реальный погодный сервис.

> #### 🧠 Ключевая концепция: Докстринги критически важны!
>
> LLM агента в значительной степени полагается на **докстринг** функции, чтобы понять:
>
> *   *Что* делает инструмент.
> *   *Когда* его использовать.
> *   *Какие аргументы* он требует (`city: str`).
> *   *Какую информацию* он возвращает.
>
> **💡 Лучшая практика:** Пишите ясные, описательные и точные докстринги для ваших инструментов. Это необходимо для того, чтобы LLM правильно использовал инструмент.


In [5]:
# @title Определение инструмента get_weather
def get_weather(city: str) -> dict:
    """
    Description:
    ---------------
        Получает текущий прогноз погоды для указанного города.

    Args:
    ---------------
        city (str): Название города (например, "Нью-Йорк", "Лондон", "Токио").

    Returns:
    ---------------
        dict: Словарь, содержащий информацию о погоде.
              Включает ключ 'status' ('success' или 'error').
              При успехе ('success') содержит ключ 'report' с деталями погоды.
              При ошибке ('error') содержит ключ 'error_message'.
    """
    # Логирование вызова инструмента
    print(f"--- Инструмент: get_weather вызван для города: {city} ---")
    # Базовая нормализация для обработки разных вариантов написания
    city_normalized = city.lower().replace(" ", "")

    # Имитация базы данных с прогнозами погоды
    mock_weather_db = {
        "ньюйорк": {"status": "success", "report": "В Нью-Йорке солнечно, температура 25°C."},
        "лондон": {"status": "success", "report": "В Лондоне облачно, температура 15°C."},
        "токио": {"status": "success", "report": "В Токио небольшой дождь, температура 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Извините, у меня нет информации о погоде для города '{city}'."}

# Пример использования инструмента (необязательный тест)
print(get_weather("Нью-Йорк"))
print(get_weather("Париж"))

--- Инструмент: get_weather вызван для города: Нью-Йорк ---
{'status': 'error', 'error_message': "Извините, у меня нет информации о погоде для города 'Нью-Йорк'."}
--- Инструмент: get_weather вызван для города: Париж ---
{'status': 'error', 'error_message': "Извините, у меня нет информации о погоде для города 'Париж'."}


### 🧠 2. Определяем агента (`weather_agent`)

Теперь создадим самого **агента (Agent)**. В ADK `Agent` оркестрирует взаимодействие между пользователем, LLM и доступными инструментами.

Мы настраиваем его с помощью нескольких ключевых параметров:

*   `name`: Уникальный идентификатор для этого агента (например, `"weather_agent_v1"`).
*   `model`: Указывает, какую LLM использовать (например, `MODEL_GEMINI_2_0_FLASH`). Мы начнем с конкретной модели Gemini.
*   `description`: Краткое описание общей цели агента. Это становится критически важным позже, когда другим агентам нужно будет решить, делегировать ли задачи *этому* агенту.
*   `instruction`: Подробное руководство для LLM о том, как себя вести, какова его роль, цели и, в частности, *как и когда* использовать назначенные ему `tools`.
*   `tools`: Список, содержащий фактические функции-инструменты Python, которые агенту разрешено использовать (например, `[get_weather]`).

> **💡 Лучшая практика:** Давайте четкие и конкретные инструкции (`instruction`). Чем детальнее инструкции, тем лучше LLM сможет понять свою роль и как эффективно использовать свои инструменты. При необходимости будьте точны в описании обработки ошибок.

> **💡 Лучшая практика:** Выбирайте описательные `name` и `description`. Они используются внутри ADK и жизненно важны для таких функций, как автоматическое делегирование (рассмотрим позже).


In [6]:
# @title Определение Агента Погоды
# Используем одну из ранее определенных констант моделей
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Начинаем с Gemini

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Может быть строкой для Gemini или объектом LiteLlm
    description="Предоставляет информацию о погоде для конкретных городов.",
    instruction="Вы — полезный погодный ассистент. "
                "Когда пользователь спрашивает о погоде в конкретном городе, "
                "используйте инструмент 'get_weather', чтобы найти информацию. "
                "Если инструмент возвращает ошибку, вежливо сообщите об этом пользователю. "
                "Если инструмент отработал успешно, четко представьте отчет о погоде.",
    tools=[get_weather], # Передаем саму функцию напрямую
)

print(f"Агент '{weather_agent.name}' создан с использованием модели '{AGENT_MODEL}'.")

Агент 'weather_agent_v1' создан с использованием модели 'gemini-2.0-flash'.


### ⚙️ 3. Настраиваем Runner и Session Service

Для управления диалогами и выполнения агента нам понадобятся еще два компонента:

*   `SessionService`: Отвечает за управление историей разговоров и состоянием для разных пользователей и сессий. `InMemorySessionService` — это простая реализация, которая хранит все в памяти и подходит для тестирования и простых приложений. Она отслеживает обмен сообщениями. Мы подробнее рассмотрим сохранение состояния в Шаге 4.
*   `Runner`: Движок, который оркестрирует поток взаимодействия. Он принимает ввод пользователя, направляет его соответствующему агенту, управляет вызовами LLM и инструментов на основе логики агента, обрабатывает обновления сессии через `SessionService` и генерирует события, представляющие ход взаимодействия.

In [7]:
# @title Настройка сервиса сессий и Runner'а

# --- Управление сессиями ---
# Ключевая концепция: SessionService хранит историю и состояние диалога.
# InMemorySessionService — это простое, непостоянное хранилище для этого примера.
session_service = InMemorySessionService()

# Определяем константы для идентификации контекста взаимодействия
APP_NAME = "weather_tutorial_app" # Имя приложения (лучше оставить на английском)
USER_ID = "user_1"                # ID пользователя
SESSION_ID = "session_001"        # ID сессии (используем фиксированный для простоты)

# Создаем конкретную сессию, в которой будет происходить диалог
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Сессия создана: Приложение='{APP_NAME}', Пользователь='{USER_ID}', Сессия='{SESSION_ID}'")

# --- Runner (Исполнитель) ---
# Ключевая концепция: Runner управляет циклом выполнения агента.
runner = Runner(
    agent=weather_agent,         # Агент, которого мы хотим запустить
    app_name=APP_NAME,           # Связывает запуски с нашим приложением
    session_service=session_service # Использует наш менеджер сессий
)
print(f"Runner создан для агента '{runner.agent.name}'.")

Сессия создана: Приложение='weather_tutorial_app', Пользователь='user_1', Сессия='session_001'
Runner создан для агента 'weather_agent_v1'.


### 💬 4. Взаимодействуем с агентом

Нам нужен способ отправлять сообщения нашему агенту и получать его ответы. Поскольку вызовы LLM и выполнение инструментов могут занимать время, `Runner` в ADK работает асинхронно.

Мы определим асинхронную вспомогательную функцию (`call_agent_async`), которая:

1.  Принимает строку запроса от пользователя.
2.  Упаковывает ее в формат `Content` ADK.
3.  Вызывает `runner.run_async`, предоставляя контекст пользователя/сессии и новое сообщение.
4.  Итерирует по **событиям (Events)**, генерируемым `Runner`. События представляют собой шаги выполнения агента (например, запрошен вызов инструмента, получен результат инструмента, промежуточная мысль LLM, финальный ответ).
5.  Определяет и выводит событие **финального ответа**, используя `event.is_final_response()`.

> #### 🤔 Зачем `async`?
>
> Взаимодействия с LLM и потенциально с инструментами (например, внешними API) являются операциями, связанными с вводом-выводом (I/O-bound). Использование `asyncio` позволяет программе эффективно обрабатывать эти операции, не блокируя выполнение.

In [8]:
# @title Определение функции для взаимодействия с агентом

from google.genai import types # Для создания контента/частей сообщения

async def call_agent_async(query: str, runner, user_id, session_id):
    """
    Description:
    ---------------
        Отправляет запрос агенту и выводит его итоговый ответ в консоль.

    Args:
    ---------------
        query (str): Входной запрос от пользователя.
        runner: Экземпляр Runner, который выполняет логику агента.
        user_id (str): Идентификатор пользователя.
        session_id (str): Идентификатор текущей сессии.

    Returns:
    ---------------
        None: Функция ничего не возвращает; она выводит результат в консоль.
    """
    print(f"\n>>> Запрос пользователя: {query}")

    # Подготавливаем сообщение пользователя в формате ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    # Ответ по умолчанию, если агент не вернет итогового сообщения
    final_response_text = "Агент не предоставил итогового ответа."

    # Ключевая концепция: run_async выполняет логику агента и возвращает (yields) События.
    # Мы перебираем события, чтобы найти итоговый ответ.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Вы можете раскомментировать строку ниже, чтобы увидеть *все* события во время выполнения
        # print(f"  [Событие] Автор: {event.author}, Тип: {type(event).__name__}, Итоговый: {event.is_final_response()}, Содержимое: {event.content}")

        # Ключевая концепция: is_final_response() помечает итоговое сообщение для данного шага диалога.
        if event.is_final_response():
            if event.content and event.content.parts:
               # Предполагаем, что текстовый ответ находится в первой части
               final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Обработка возможных ошибок/эскалаций
               final_response_text = f"Агент выполнил эскалацию: {event.error_message or 'Конкретное сообщение отсутствует.'}"
            # При необходимости добавьте сюда дополнительные проверки (например, по кодам ошибок)
            break # Прерываем обработку событий, как только найден итоговый ответ

    print(f"<<< Ответ агента: {final_response_text}")

### ▶️ 5. Запускаем диалог

Наконец, давайте протестируем нашу настройку, отправив несколько запросов агенту. Мы обернем наши асинхронные вызовы в основную `async` функцию и запустим ее с помощью `await`.

Следите за выводом:

*   Смотрите на запросы пользователя.
*   Обратите внимание на логи `--- Tool: get_weather called... ---`, когда агент использует инструмент.
*   Наблюдайте за финальными ответами агента, включая то, как он обрабатывает случай, когда данные о погоде недоступны (для Парижа).

In [11]:
# @title Запуск начального диалога

# Создаем асинхронную функцию для запуска диалога
async def run_conversation():
    await call_agent_async("Какая погода в Лондоне?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    # Ожидаем сообщение об ошибке от инструмента, так как "Париж" не входит в нашу имитацию базы данных
    await call_agent_async("А как насчет Парижа?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("Расскажи мне о погоде в Нью-Йорке",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Выполняем диалог, используя await в асинхронном контексте (например, в Colab/Jupyter)
await run_conversation()

# --- ИЛИ ---

# Раскомментируйте следующие строки, если запускаете код как стандартный Python-скрипт (.py файл):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"Произошла ошибка: {e}")


>>> Запрос пользователя: Какая погода в Лондоне?
--- Инструмент: get_weather вызван для города: Лондон ---
<<< Ответ агента: В Лондоне облачно, температура 15°C.

>>> Запрос пользователя: А как насчет Парижа?
--- Инструмент: get_weather вызван для города: Париж ---
<<< Ответ агента: Извините, у меня нет информации о погоде для города Париж.

>>> Запрос пользователя: Расскажи мне о погоде в Нью-Йорке
--- Инструмент: get_weather вызван для города: Нью-Йорк ---
<<< Ответ агента: Извините, у меня нет информации о погоде для города Нью-Йорк.


---

Поздравляем! Вы успешно создали своего первого агента в ADK и повзаимодействовали с ним. Он понимает запрос пользователя, использует инструмент для поиска информации и адекватно отвечает на основе результата работы инструмента.

В следующем шаге мы рассмотрим, как легко можно сменить языковую модель, лежащую в основе этого агента.

## 🧠 Шаг 2: Мультимодельный подход с LiteLLM [Опционально]

На Шаге 1 мы создали функционального погодного агента, работающего на конкретной модели Gemini. Хотя это и эффективно, реальные приложения часто выигрывают от гибкости использования *различных* больших языковых моделей (LLM). Почему?

*   **Производительность:** Некоторые модели превосходно справляются с определенными задачами (например, написание кода, рассуждения, творчество).
*   **Стоимость:** У разных моделей разные ценовые категории.
*   **Возможности:** Модели предлагают разнообразные функции, размеры контекстного окна и опции для дообучения (fine-tuning).
*   **Доступность/Резервирование:** Наличие альтернатив гарантирует, что ваше приложение останется работоспособным, даже если у одного из провайдеров возникнут проблемы.

ADK делает переключение между моделями бесшовным благодаря интеграции с библиотекой [**LiteLLM**](https.github.com/BerriAI/litellm). LiteLLM выступает в роли единого интерфейса для более чем 100 различных LLM.

### ✅ На этом шаге мы:

1.  Узнаем, как настроить `Agent` в ADK для использования моделей от провайдеров, таких как OpenAI (GPT) и Anthropic (Claude), с помощью обертки `LiteLlm`.
2.  Определим, настроим (с их собственными сессиями и `Runner`-ами) и сразу же протестируем экземпляры нашего погодного агента, каждый из которых работает на своей LLM.
3.  Повзаимодействуем с этими разными агентами, чтобы увидеть потенциальные различия в их ответах, даже при использовании одного и того же инструмента.

### ⚙️ 1. Импортируем `LiteLlm`

Мы уже импортировали его при начальной настройке (Шаг 0), но это ключевой компонент для поддержки нескольких моделей.

In [ ]:
# @title 1. Import LiteLlm
from google.adk.models.lite_llm import LiteLlm

### 🤖 2. Определяем и тестируем мультимодельных агентов

Вместо того чтобы передавать только строку с именем модели (которая по умолчанию указывает на модели Gemini от Google), мы оборачиваем желаемый идентификатор модели в класс `LiteLlm`.

> #### 🧠 Ключевая концепция: Обертка `LiteLlm`
>
> Синтаксис `LiteLlm(model="provider/model_name")` указывает ADK направлять запросы для этого агента через библиотеку LiteLLM к указанному провайдеру моделей.

Убедитесь, что вы настроили необходимые API-ключи для OpenAI и Anthropic на Шаге 0. Мы будем использовать ранее определенную функцию `call_agent_async` (которая теперь принимает `runner`, `user_id` и `session_id`) для взаимодействия с каждым агентом сразу после его настройки.

Каждый блок кода ниже будет:

*   Определять агента с использованием конкретной модели LiteLLM (`MODEL_GPT_4O` или `MODEL_CLAUDE_SONNET`).
*   Создавать *новый, отдельный* `InMemorySessionService` и сессию специально для тестового запуска этого агента. Это позволяет изолировать истории диалогов для демонстрации.
*   Создавать `Runner`, настроенный для конкретного агента и его сервиса сессий.
*   Немедленно вызывать `call_agent_async` для отправки запроса и тестирования агента.

> **💡 Лучшая практика:** Используйте константы для имен моделей (такие как `MODEL_GPT_4O`, `MODEL_CLAUDE_SONNET`, определенные на Шаге 0), чтобы избежать опечаток и упростить управление кодом.

> **🛡️ Обработка ошибок:** Мы оборачиваем определения агентов в блоки `try...except`. Это предотвращает сбой всей ячейки кода, если API-ключ для конкретного провайдера отсутствует или недействителен, позволяя туториалу продолжаться с теми моделями, которые *настроены*.

Сначала создадим и протестируем агента, использующего GPT-4o от OpenAI.

In [ ]:
# @title Определение и тестирование GPT-агента

# Убедитесь, что функция 'get_weather' из Шага 1 определена в вашем окружении.
# Убедитесь, что функция 'call_agent_async' определена ранее.

# --- Агент, использующий GPT-4o ---
weather_agent_gpt = None # Инициализируем как None
runner_gpt = None      # Инициализируем Runner как None

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # Ключевое изменение: Оборачиваем идентификатор модели в LiteLlm
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Предоставляет информацию о погоде (используя GPT-4o).",
        instruction="Вы — полезный погодный ассистент, работающий на GPT-4o. "
                    "Используйте инструмент 'get_weather' для запросов о погоде в городах. "
                    "В зависимости от статуса ответа инструмента, четко представьте успешный отчет или вежливое сообщение об ошибке.",
        tools=[get_weather], # Используем тот же инструмент повторно
    )
    print(f"Агент '{weather_agent_gpt.name}' создан с использованием модели '{MODEL_GPT_4O}'.")

    # InMemorySessionService — это простое, непостоянное хранилище для этого примера.
    session_service_gpt = InMemorySessionService() # Создаем выделенный сервис сессий

    # Определяем константы для идентификации контекста взаимодействия
    APP_NAME_GPT = "weather_tutorial_app_gpt" # Уникальное имя приложения для этого теста
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # Используем фиксированный ID для простоты

    # Создаем конкретную сессию, в которой будет происходить диалог
    session_gpt = await session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Сессия создана: Приложение='{APP_NAME_GPT}', Пользователь='{USER_ID_GPT}', Сессия='{SESSION_ID_GPT}'")

    # Создаем Runner специально для этого агента и его сервиса сессий
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # Используем специфичное имя приложения
        session_service=session_service_gpt # Используем специфичный сервис сессий
        )
    print(f"Runner создан для агента '{runner_gpt.agent.name}'.")

    # --- Тестирование GPT-агента ---
    print("\n--- Тестирование GPT-агента ---")
    # Убедимся, что call_agent_async использует правильные runner, user_id и session_id
    await call_agent_async(query = "Какая погода в Токио?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)
    # --- ИЛИ ---

    # Раскомментируйте следующие строки, если запускаете код как стандартный Python-скрипт (.py файл):
    # import asyncio
    # if __name__ == "__main__":
    #     try:
    #         asyncio.run(call_agent_async(query = "Какая погода в Токио?",
    #                      runner=runner_gpt,
    #                      user_id=USER_ID_GPT,
    #                      session_id=SESSION_ID_GPT))
    #     except Exception as e:
    #         print(f"Произошла ошибка: {e}")

except Exception as e:
    print(f"❌ Не удалось создать или запустить GPT-агента '{MODEL_GPT_4O}'. Проверьте API-ключ и название модели. Ошибка: {e}")

Далее мы сделаем то же самое для Клода Соннета из Anthropic.

In [ ]:
# @title Определение и тестирование Claude-агента

# Убедитесь, что функция 'get_weather' из Шага 1 определена в вашем окружении.
# Убедитесь, что функция 'call_agent_async' определена ранее.

# --- Агент, использующий Claude Sonnet ---
weather_agent_claude = None # Инициализируем как None
runner_claude = None      # Инициализируем Runner как None

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # Ключевое изменение: Оборачиваем идентификатор модели в LiteLlm
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Предоставляет информацию о погоде (используя Claude Sonnet).",
        instruction="Вы — полезный погодный ассистент, работающий на Claude Sonnet. "
                    "Используйте инструмент 'get_weather' для запросов о погоде в городах. "
                    "Анализируйте словарь, возвращаемый инструментом ('status', 'report'/'error_message'). "
                    "Четко представьте успешный отчет или вежливое сообщение об ошибке.",
        tools=[get_weather], # Используем тот же инструмент повторно
    )
    print(f"Агент '{weather_agent_claude.name}' создан с использованием модели '{MODEL_CLAUDE_SONNET}'.")

    # InMemorySessionService — это простое, непостоянное хранилище для этого примера.
    session_service_claude = InMemorySessionService() # Создаем выделенный сервис сессий

    # Определяем константы для идентификации контекста взаимодействия
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # Уникальное имя приложения
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # Используем фиксированный ID для простоты

    # Создаем конкретную сессию, в которой будет происходить диалог
    session_claude = await session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Сессия создана: Приложение='{APP_NAME_CLAUDE}', Пользователь='{USER_ID_CLAUDE}', Сессия='{SESSION_ID_CLAUDE}'")

    # Создаем Runner специально для этого агента и его сервиса сессий
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # Используем специфичное имя приложения
        session_service=session_service_claude # Используем специфичный сервис сессий
        )
    print(f"Runner создан для агента '{runner_claude.agent.name}'.")

    # --- Тестирование Claude-агента ---
    print("\n--- Тестирование Claude-агента ---")
    # Убедимся, что call_agent_async использует правильные runner, user_id и session_id
    await call_agent_async(query = "Погода в Лондоне, пожалуйста.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

    # --- ИЛИ ---

    # Раскомментируйте следующие строки, если запускаете код как стандартный Python-скрипт (.py файл):
    # import asyncio
    # if __name__ == "__main__":
    #     try:
    #         asyncio.run(call_agent_async(query = "Погода в Лондоне, пожалуйста.",
    #                      runner=runner_claude,
    #                      user_id=USER_ID_CLAUDE,
    #                      session_id=SESSION_ID_CLAUDE))
    #     except Exception as e:
    #         print(f"Произошла ошибка: {e}")


except Exception as e:
    print(f"❌ Не удалось создать или запустить Claude-агента '{MODEL_CLAUDE_SONNET}'. Проверьте API-ключ и название модели. Ошибка: {e}")

Сначала создадим и протестируем агента, использующего GPT-4o от OpenAI. Внимательно наблюдайте за выводом из обоих блоков кода. Вы должны увидеть:

1.  Каждый агент (`weather_agent_gpt`, `weather_agent_claude`) успешно создается (если API-ключи действительны).
2.  Для каждого из них настраивается выделенная сессия и `Runner`.
3.  Каждый агент правильно определяет необходимость использования инструмента `get_weather` при обработке запроса (вы увидите лог `--- Tool: get_weather called... ---`).
4.  *Базовая логика инструмента* остается идентичной, всегда возвращая наши имитационные данные.
5.  Однако **финальный текстовый ответ**, сгенерированный каждым агентом, может незначительно отличаться по формулировкам, тону или форматированию. Это происходит потому, что инструкция интерпретируется и выполняется разными LLM (GPT-4o против Claude Sonnet).

Этот шаг демонстрирует мощь и гибкость, которые предоставляют ADK и LiteLLM. Вы можете легко экспериментировать и развертывать агентов, используя различные LLM, сохраняя при этом вашу основную логику приложения (инструменты, фундаментальную структуру агента) неизменной.

В следующем шаге мы выйдем за рамки одного агента и создадим небольшую команду, в которой агенты смогут делегировать задачи друг другу!

---

## 🤝 Шаг 3: Создание команды агентов — делегирование для приветствий и прощаний

На Шагах 1 и 2 мы создали и поэкспериментировали с одним агентом, сфокусированным исключительно на поиске погоды. Хотя он и эффективен для своей конкретной задачи, реальные приложения часто требуют обработки более широкого спектра взаимодействий с пользователем. Мы *могли бы* продолжать добавлять больше инструментов и сложных инструкций нашему единственному погодному агенту, но это быстро может стать неуправляемым и менее эффективным.

Более надежный подход — создать **команду агентов**. Это включает в себя:

1.  Создание нескольких **специализированных агентов**, каждый из которых предназначен для определенной возможности (например, один для погоды, один для приветствий, один для вычислений).
2.  Назначение **корневого агента** (или оркестратора), который получает первоначальный запрос пользователя.
3.  Предоставление корневому агенту возможности **делегировать** запрос наиболее подходящему специализированному подагенту на основе намерения пользователя.

### 🤔 Зачем создавать команду агентов?

*   **Модульность:** Проще разрабатывать, тестировать и поддерживать отдельных агентов.
*   **Специализация:** Каждый агент может быть точно настроен (инструкции, выбор модели) для своей конкретной задачи.
*   **Масштабируемость:** Проще добавлять новые возможности, добавляя новых агентов.
*   **Эффективность:** Позволяет использовать потенциально более простые/дешевые модели для более простых задач (например, приветствий).

### ✅ На этом шаге мы:

1.  Определим простые инструменты для обработки приветствий (`say_hello`) и прощаний (`say_goodbye`).
2.  Создадим двух новых специализированных подагентов: `greeting_agent` и `farewell_agent`.
3.  Обновим нашего основного погодного агента (`weather_agent_v2`), чтобы он действовал как **корневой агент**.
4.  Настроим корневого агента с его подагентами, включив **автоматическое делегирование**.
5.  Протестируем поток делегирования, отправляя различные типы запросов корневому агенту.

### 🛠️ 1. Определяем инструменты для подагентов

Сначала создадим простые Python-функции, которые будут служить инструментами для наших новых специализированных агентов. Помните, что четкие докстринги жизненно важны для агентов, которые будут их использовать.

In [ ]:
# @title Определение инструментов для приветствия и прощания
from typing import Optional # Убедитесь, что импортирован Optional

# Убедитесь, что функция 'get_weather' из Шага 1 доступна, если запускаете этот шаг отдельно.
# def get_weather(city: str) -> dict: ... (из Шага 1)

def say_hello(name: Optional[str] = None) -> str: # ИЗМЕНЕННАЯ СИГНАТУРА
    """
    Description:
    ---------------
        Возвращает простое приветствие. Если указано имя, оно будет использовано в приветствии.

    Args:
    ---------------
        name (str, optional): Имя человека для приветствия. Если не указано, используется общее приветствие.

    Returns:
    ---------------
        str: Дружелюбное приветственное сообщение.
    """
    # НАЧАЛО ИЗМЕНЕНИЯ
    if name:
        greeting = f"Здравствуйте, {name}!"
        print(f"--- Инструмент: say_hello вызван с именем: {name} ---")
    else:
        greeting = "Здравствуйте!" # Приветствие по умолчанию, если имя не передано или равно None
        print(f"--- Инструмент: say_hello вызван без конкретного имени (значение аргумента name: {name}) ---")
    return greeting
    # КОНЕЦ ИЗМЕНЕНИЯ

def say_goodbye() -> str:
    """
    Description:
    ---------------
        Возвращает простое прощальное сообщение для завершения диалога.

    Args:
    ---------------
        Нет.

    Returns:
    ---------------
        str: Прощальное сообщение.
    """
    print(f"--- Инструмент: say_goodbye вызван ---")
    return "До свидания! Хорошего дня."

print("Инструменты для приветствия и прощания определены.")

# Необязательный самотест
print(say_hello("Алиса"))
print(say_hello())          # Тест без аргумента (должно использоваться приветствие по умолчанию)
print(say_hello(name=None)) # Тест с явной передачей None (должно использоваться приветствие по умолчанию)

Инструменты для приветствия и прощания определены.
--- Инструмент: say_hello вызван с именем: Алиса ---
Здравствуйте, Алиса!
--- Инструмент: say_hello вызван без конкретного имени (значение аргумента name: None) ---
Здравствуйте!
--- Инструмент: say_hello вызван без конкретного имени (значение аргумента name: None) ---
Здравствуйте!


---

### 🤖 2. Определяем подагентов (для приветствий и прощаний)

Теперь создадим экземпляры `Agent` для наших специалистов. Обратите внимание на их узкоспециализированные инструкции (`instruction`) и, что критически важно, на их четкое описание (`description`). `description` — это основная информация, которую *корневой агент* использует, чтобы решить, *когда* делегировать задачи этим подагентам.

> **💡 Лучшая практика:** Поле `description` у подагента должно точно и кратко описывать его конкретную возможность. Это крайне важно для эффективного автоматического делегирования.

> **💡 Лучшая практика:** Поле `instruction` у подагента должно быть адаптировано к его ограниченной сфере деятельности, точно указывая, что ему делать и *чего не делать* (например, «Твоя *единственная* задача — ...»).

In [15]:
# @title Определение субагентов для приветствия и прощания

# Если вы хотите использовать модели, отличные от Gemini, убедитесь, что LiteLlm импортирован, а ключи API установлены (из Шага 0/2)
# from google.adk.models.lite_llm import LiteLlm
# Константы MODEL_GPT_4O, MODEL_CLAUDE_SONNET и т.д. должны быть определены
# В противном случае продолжайте использовать: model = MODEL_GEMINI_2_0_FLASH

# --- Агент для приветствий ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # Используем потенциально другую/более дешевую модель для простой задачи
        model = MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O), # Если вы хотите поэкспериментировать с другими моделями
        name="greeting_agent",
        instruction="Вы — Агент для приветствий. Ваша ЕДИНСТВЕННАЯ задача — предоставить пользователю дружелюбное приветствие. "
                    "Используйте инструмент 'say_hello' для генерации приветствия. "
                    "Если пользователь указывает свое имя, обязательно передайте его в инструмент. "
                    "Не вступайте в другие разговоры и не выполняйте другие задачи.",
        description="Обрабатывает простые приветствия, используя инструмент 'say_hello'.", # Критически важно для делегирования
        tools=[say_hello],
    )
    print(f"✅ Агент '{greeting_agent.name}' создан с использованием модели '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Не удалось создать Агента для приветствий. Проверьте API-ключ ({greeting_agent.model}). Ошибка: {e}")

# --- Агент для прощаний ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # Можно использовать ту же или другую модель
        model = MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O), # Если вы хотите поэкспериментировать с другими моделями
        name="farewell_agent",
        instruction="Вы — Агент для прощаний. Ваша ЕДИНСТВЕННАЯ задача — предоставить вежливое прощальное сообщение. "
                    "Используйте инструмент 'say_goodbye', когда пользователь дает понять, что уходит или завершает разговор "
                    "(например, используя слова 'пока', 'до свидания', 'спасибо, пока', 'увидимся'). "
                    "Не выполняйте никаких других действий.",
        description="Обрабатывает простые прощания, используя инструмент 'say_goodbye'.", # Критически важно для делегирования
        tools=[say_goodbye],
    )
    print(f"✅ Агент '{farewell_agent.name}' создан с использованием модели '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Не удалось создать Агента для прощаний. Проверьте API-ключ ({farewell_agent.model}). Ошибка: {e}")

✅ Агент 'greeting_agent' создан с использованием модели 'gemini-2.0-flash'.
✅ Агент 'farewell_agent' создан с использованием модели 'gemini-2.0-flash'.


---

### 🌳 3. Определяем корневого агента (Weather Agent v2) с подагентами

Теперь мы обновляем нашего `weather_agent`. Ключевые изменения:

*   Добавление параметра `sub_agents`: Мы передаем список, содержащий созданные нами экземпляры `greeting_agent` и `farewell_agent`.
*   Обновление `instruction`: Мы явно сообщаем корневому агенту *о его подагентах* и *когда* ему следует делегировать им задачи.

> #### 🧠 Ключевая концепция: Автоматическое делегирование (Auto Flow)
>
> Предоставляя список `sub_agents`, ADK включает автоматическое делегирование. Когда корневой агент получает запрос пользователя, его LLM рассматривает не только собственные инструкции и инструменты, но и `description` каждого подагента. Если LLM определяет, что запрос лучше соответствует описанным возможностям подагента (например, «Обрабатывает простые приветствия»), он автоматически генерирует специальное внутреннее действие для *передачи управления* этому подагенту на данный ход. Затем подагент обрабатывает запрос, используя свою собственную модель, инструкции и инструменты.

> **💡 Лучшая практика:** Убедитесь, что инструкции корневого агента четко направляют его решения о делегировании. Упомяните подагентов по имени и опишите условия, при которых должно происходить делегирование.

In [16]:
# @title Определение Корневого Агента с Субагентами

# Перед определением корневого агента убедитесь, что субагенты были успешно созданы.
# Также убедитесь, что определен оригинальный инструмент 'get_weather'.
root_agent = None
runner_root = None # Инициализируем Runner

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # Будем использовать производительную модель Gemini для корневого агента, чтобы он справлялся с координацией
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2", # Даем ему новое имя версии
        model=root_agent_model,
        description="Главный агент-координатор. Обрабатывает запросы о погоде и делегирует приветствия/прощания специалистам.",
        instruction="Вы — главный Агент Погоды, координирующий команду. Ваша основная обязанность — предоставлять информацию о погоде. "
                    "Используйте инструмент 'get_weather' ТОЛЬКО для конкретных запросов о погоде (например, 'погода в Лондоне'). "
                    "У вас есть специализированные субагенты: "
                    "1. 'greeting_agent': Обрабатывает простые приветствия, такие как 'Привет', 'Здравствуйте'. Делегируйте ему эти задачи. "
                    "2. 'farewell_agent': Обрабатывает простые прощания, такие как 'Пока', 'Увидимся'. Делегируйте ему эти задачи. "
                    "Анализируйте запрос пользователя. Если это приветствие, делегируйте 'greeting_agent'. Если это прощание, делегируйте 'farewell_agent'. "
                    "Если это запрос о погоде, обработайте его самостоятельно с помощью 'get_weather'. "
                    "На все остальное отвечайте соответствующим образом или сообщите, что не можете обработать запрос.",
        tools=[get_weather], # Корневому агенту все еще нужен инструмент погоды для его основной задачи
        # Ключевое изменение: подключаем субагентов здесь!
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Корневой Агент '{weather_agent_team.name}' создан с использованием модели '{root_agent_model}' и субагентами: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Невозможно создать корневого агента, так как один или несколько субагентов не были инициализированы или отсутствует инструмент 'get_weather'.")
    if not greeting_agent: print(" - Отсутствует Агент для приветствий.")
    if not farewell_agent: print(" - Отсутствует Агент для прощаний.")
    if 'get_weather' not in globals(): print(" - Отсутствует функция get_weather.")

✅ Корневой Агент 'weather_agent_v2' создан с использованием модели 'gemini-2.0-flash' и субагентами: ['greeting_agent', 'farewell_agent']


---

### 💬 4. Взаимодействуем с командой агентов

Теперь, когда мы определили нашего корневого агента (`weather_agent_team`) с его специализированными подагентами, давайте протестируем механизм делегирования.

Следующий блок кода:

1.  Определит асинхронную функцию `run_team_conversation`.
2.  Внутри этой функции создаст *новый, выделенный* `InMemorySessionService` и специальную сессию (`session_001_agent_team`) только для этого тестового запуска. Это изолирует историю диалога для тестирования динамики команды.
3.  Создаст `Runner` (`runner_agent_team`), настроенный на использование нашего `weather_agent_team` (корневого агента) и выделенного сервиса сессий.
4.  Использует нашу обновленную функцию `call_agent_async` для отправки различных типов запросов (приветствие, запрос погоды, прощание) в `runner_agent_team`. Мы явно передаем `runner`, ID пользователя и ID сессии для этого конкретного теста.
5.  Немедленно выполнит функцию `run_team_conversation`.

Мы ожидаем следующий поток выполнения:

1.  Запрос «Hello there!» поступает в `runner_agent_team`.
2.  Корневой агент (`weather_agent_team`) получает его и, основываясь на своих инструкциях и описании `greeting_agent`, делегирует задачу.
3.  `greeting_agent` обрабатывает запрос, вызывает свой инструмент `say_hello` и генерирует ответ.
4.  Запрос «What is the weather in New York?» *не делегируется* и обрабатывается непосредственно корневым агентом с использованием его инструмента `get_weather`.
5.  Запрос «Thanks, bye!» делегируется `farewell_agent`, который использует свой инструмент `say_goodbye`.

In [17]:
# @title Взаимодействие с командой агентов
import asyncio # Убедитесь, что asyncio импортирован

# Убедитесь, что корневой агент (например, 'weather_agent_team' из предыдущей ячейки) определен.
# Убедитесь, что функция call_agent_async определена.

# Проверяем, существует ли переменная корневого агента, перед определением функции диалога
root_agent_var_name = 'root_agent' # Имя по умолчанию из руководства
if 'weather_agent_team' in globals(): # Проверяем, использовал ли пользователь это имя
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Корневой агент ('root_agent' или 'weather_agent_team') не найден. Невозможно определить run_team_conversation.")
    # Присваиваем фиктивное значение, чтобы избежать NameError, если блок все равно будет запущен
    root_agent = None

# Определяем и запускаем диалог, только если корневой агент существует
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # Определяем основную асинхронную функцию для логики диалога.
    # Ключевые слова 'await' ВНУТРИ этой функции необходимы для асинхронных операций.
    async def run_team_conversation():
        print("\n--- Тестирование делегирования в команде агентов ---")
        session_service = InMemorySessionService()
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Сессия создана: Приложение='{APP_NAME}', Пользователь='{USER_ID}', Сессия='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner(
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner создан для агента '{actual_root_agent.name}'.")

        # --- Взаимодействия с использованием await (корректно внутри async def) ---
        await call_agent_async(query = "Привет!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Какая погода в Нью-Йорке?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Спасибо, пока!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # --- Выполнение асинхронной функции `run_team_conversation` ---
    # Выберите ОДИН из методов ниже в зависимости от вашей среды.
    # Примечание: Для этого могут потребоваться API-ключи для используемых моделей!

    # МЕТОД 1: Прямой await (По умолчанию для Notebooks/Async REPL)
    # Если ваша среда поддерживает top-level await (как в Colab/Jupyter),
    # это означает, что цикл событий уже запущен, и вы можете напрямую вызвать функцию через await.
    print("Попытка выполнения через 'await' (по умолчанию для ноутбуков)...")
    await run_team_conversation()

    # МЕТОД 2: asyncio.run (Для стандартных Python-скриптов [.py])
    # Если вы запускаете этот код как стандартный Python-скрипт из терминала,
    # контекст скрипта является синхронным. `asyncio.run()` необходим для
    # создания и управления циклом событий для выполнения вашей асинхронной функции.
    # Чтобы использовать этот метод:
    # 1. Закомментируйте строку `await run_team_conversation()` выше.
    # 2. Раскомментируйте следующий блок:
    """
    import asyncio
    if __name__ == "__main__": # Гарантирует, что код запустится только при прямом выполнении скрипта
        print("Выполнение через 'asyncio.run()' (для стандартных Python-скриптов)...")
        try:
            # Эта команда создает цикл событий, запускает вашу асинхронную функцию и закрывает цикл.
            asyncio.run(run_team_conversation())
        except Exception as e:
            print(f"Произошла ошибка: {e}")
    """

else:
    # Это сообщение выводится, если переменная корневого агента не была найдена ранее
    print("\n⚠️ Пропускаем выполнение диалога с командой агентов, так как корневой агент не был успешно определен на предыдущем шаге.")

Попытка выполнения через 'await' (по умолчанию для ноутбуков)...

--- Тестирование делегирования в команде агентов ---
Сессия создана: Приложение='weather_tutorial_agent_team', Пользователь='user_1_agent_team', Сессия='session_001_agent_team'
Runner создан для агента 'weather_agent_v2'.

>>> Запрос пользователя: Привет!
--- Инструмент: say_hello вызван без конкретного имени (значение аргумента name: None) ---
<<< Ответ агента: Здравствуйте!


>>> Запрос пользователя: Какая погода в Нью-Йорке?
--- Инструмент: get_weather вызван для города: Нью-Йорк ---
<<< Ответ агента: Извините, у меня нет информации о погоде для города 'Нью-Йорк'.


>>> Запрос пользователя: Спасибо, пока!
--- Инструмент: say_goodbye вызван ---
<<< Ответ агента: До свидания! Хорошего дня.



---

Внимательно посмотрите на логи вывода, особенно на сообщения `--- Tool: ... called ---`. Вы должны заметить:

*   Для «Hello there!» был вызван инструмент `say_hello` (что указывает на то, что его обработал `greeting_agent`).
*   Для «What is the weather in New York?» был вызван инструмент `get_weather` (что указывает на то, что его обработал корневой агент).
*   Для «Thanks, bye!» был вызван инструмент `say_goodbye` (что указывает на то, что его обработал `farewell_agent`).

Это подтверждает успешное **автоматическое делегирование**! Корневой агент, руководствуясь своими инструкциями и `description` своих `sub_agents`, правильно маршрутизировал запросы пользователей к соответствующему специализированному агенту в команде.

Вы только что структурировали свое приложение с несколькими взаимодействующими агентами. Этот модульный дизайн является основополагающим для создания более сложных и функциональных агентных систем. На следующем шаге мы дадим нашим агентам возможность запоминать информацию между ходами диалога с помощью состояния сессии.

## 💾 Шаг 4: Добавление памяти и персонализации с помощью состояния сессии (Session State)

До сих пор наша команда агентов могла обрабатывать различные задачи через делегирование, но каждое взаимодействие начиналось с чистого листа — у агентов не было памяти о прошлых разговорах или предпочтениях пользователя в рамках одной сессии. Для создания более сложных и контекстно-зависимых сценариев агентам необходима **память**. ADK предоставляет эту возможность через **состояние сессии (Session State)**.

### 🤔 Что такое состояние сессии?

*   Это Python-словарь (`session.state`), привязанный к конкретной сессии пользователя (идентифицируемой по `APP_NAME`, `USER_ID`, `SESSION_ID`).
*   Он сохраняет информацию *между несколькими ходами диалога* в рамках этой сессии.
*   Агенты и инструменты могут читать и записывать данные в это состояние, что позволяет им запоминать детали, адаптировать поведение и персонализировать ответы.

### ⚙️ Как агенты взаимодействуют с состоянием:

1.  **`ToolContext` (Основной метод):** Инструменты могут принимать объект `ToolContext` (автоматически предоставляемый ADK, если он объявлен последним аргументом). Этот объект дает прямой доступ к состоянию сессии через `tool_context.state`, позволяя инструментам читать предпочтения или сохранять результаты *во время* выполнения.
2.  **`output_key` (Автосохранение ответа агента):** `Agent` можно настроить с параметром `output_key="your_key"`. Тогда ADK будет автоматически сохранять финальный текстовый ответ агента за один ход в `session.state["your_key"]`.

### ✅ На этом шаге мы улучшим нашу команду погодных ботов:

1.  Используем **новый** `InMemorySessionService` для демонстрации работы состояния в изоляции.
2.  Инициализируем состояние сессии с предпочтением пользователя `temperature_unit`.
3.  Создадим версию погодного инструмента, знающую о состоянии (`get_weather_stateful`), которая читает это предпочтение через `ToolContext` и корректирует формат вывода (Цельсий/Фаренгейт).
4.  Обновим корневого агента, чтобы он использовал этот новый инструмент, и настроим его с `output_key` для автоматического сохранения его финального отчета о погоде в состояние сессии.
5.  Проведем диалог, чтобы понаблюдать, как начальное состояние влияет на инструмент, как ручные изменения состояния влияют на последующее поведение, и как `output_key` сохраняет ответ агента.

---

### ⚙️ 1. Инициализируем новый сервис сессий и состояние

Чтобы наглядно продемонстрировать управление состоянием без влияния предыдущих шагов, мы создадим новый экземпляр `InMemorySessionService`. Мы также создадим сессию с начальным состоянием, определяющим предпочитаемую пользователем единицу измерения температуры.

In [18]:
# @title 1. Инициализация нового сервиса сессий и состояния

# Импорт необходимых компонентов для сессий
from google.adk.sessions import InMemorySessionService

# Создаем НОВЫЙ экземпляр сервиса сессий для демонстрации работы с состоянием
session_service_stateful = InMemorySessionService()
print("✅ Создан новый InMemorySessionService для демонстрации состояния.")

# Определяем НОВЫЙ ID сессии для этой части руководства
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Определяем начальные данные состояния - пользователь изначально предпочитает Цельсий
initial_state = {
    "user_preference_temperature_unit": "Цельсий"
}

# Создаем сессию, передавая начальное состояние
session_stateful = await session_service_stateful.create_session(
    app_name=APP_NAME, # Используем то же имя приложения
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< Инициализируем состояние во время создания
)
print(f"✅ Сессия '{SESSION_ID_STATEFUL}' создана для пользователя '{USER_ID_STATEFUL}'.")

# Проверяем, что начальное состояние было установлено корректно
retrieved_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Начальное состояние сессии ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Ошибка: не удалось получить сессию.")

✅ Создан новый InMemorySessionService для демонстрации состояния.
✅ Сессия 'session_state_demo_001' создана для пользователя 'user_state_demo'.

--- Начальное состояние сессии ---
{'user_preference_temperature_unit': 'Цельсий'}


---

### 🛠️ 2. Создаем инструмент, осведомленный о состоянии (`get_weather_stateful`)

Теперь создадим новую версию погодного инструмента. Его ключевая особенность — прием `tool_context: ToolContext`, что позволяет ему получать доступ к `tool_context.state`. Он будет считывать `user_preference_temperature_unit` и форматировать температуру соответствующим образом.

> #### 🧠 Ключевая концепция: `ToolContext`
>
> Этот объект является мостом, позволяющим логике вашего инструмента взаимодействовать с контекстом сессии, включая чтение и запись переменных состояния. ADK внедряет его автоматически, если он определен как последний параметр вашей функции-инструмента.

> **💡 Лучшая практика:** При чтении из состояния используйте `dictionary.get('key', default_value)`, чтобы обрабатывать случаи, когда ключ может еще не существовать. Это гарантирует, что ваш инструмент не вызовет ошибку.

In [19]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """
    Description:
    ---------------
        Получает погоду и конвертирует единицы измерения температуры на основе состояния сессии.

    Args:
    ---------------
        city (str): Название города.
        tool_context (ToolContext): Контекст инструмента, предоставляющий доступ к состоянию сессии.

    Returns:
    ---------------
        dict: Словарь с отчетом о погоде или сообщением об ошибке.
    """
    print(f"--- Инструмент: get_weather_stateful вызван для города {city} ---")

    # --- Чтение предпочтений из состояния ---
    # По умолчанию используем Цельсий, если в состоянии ничего не указано
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Цельсий")
    print(f"--- Инструмент: Чтение состояния 'user_preference_temperature_unit': {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

    # Имитация базы данных с погодой (внутренне всегда хранится в Цельсиях)
    mock_weather_db = {
        "ньюйорк": {"temp_c": 25, "condition": "солнечно"},
        "лондон": {"temp_c": 15, "condition": "облачно"},
        "токио": {"temp_c": 18, "condition": "небольшой дождь"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # Форматируем температуру в зависимости от предпочтений в состоянии
        if preferred_unit == "Фаренгейт":
            temp_value = (temp_c * 9/5) + 32 # Рассчитываем Фаренгейты
            temp_unit = "°F"
        else: # По умолчанию используем Цельсий
            temp_value = temp_c
            temp_unit = "°C"

        report = f"В городе {city.capitalize()} {condition}, температура {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"--- Инструмент: Сгенерирован отчет в {preferred_unit}. Результат: {result} ---")

        # Пример записи обратно в состояние (необязательно для этого инструмента)
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- Инструмент: Обновлено состояние 'last_city_checked_stateful': {city} ---")

        return result
    else:
        # Обработка случая, когда город не найден
        error_msg = f"Извините, у меня нет информации о погоде для города '{city}'."
        print(f"--- Инструмент: Город '{city}' не найден. ---")
        return {"status": "error", "error_message": error_msg}

print("✅ Инструмент 'get_weather_stateful', работающий с состоянием, определен.")

✅ Инструмент 'get_weather_stateful', работающий с состоянием, определен.


---

### 🌳 3. Переопределяем подагентов и обновляем корневого агента

Чтобы этот шаг был самодостаточным и строился корректно, мы сначала переопределяем `greeting_agent` и `farewell_agent` точно так же, как на Шаге 3. Затем мы определяем нашего нового корневого агента (`weather_agent_v4_stateful`):

*   Он использует новый инструмент `get_weather_stateful`.
*   Он включает подагентов для приветствий и прощаний для делегирования.
*   **Критически важно:** он устанавливает `output_key="last_weather_report"`, что автоматически сохраняет его финальный ответ о погоде в состояние сессии.

In [20]:
# @title 3. Переопределение субагентов и обновление корневого агента с output_key

# Убедитесь, что необходимые компоненты импортированы: Agent, LiteLlm, Runner
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# Убедитесь, что инструменты 'say_hello', 'say_goodbye' определены (из Шага 3)
# Убедитесь, что константы моделей MODEL_GPT_4O, MODEL_GEMINI_2_0_FLASH и т.д. определены

# --- Переопределение Агента для приветствий (из Шага 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="Вы — Агент для приветствий. Ваша ЕДИНСТВЕННАЯ задача — предоставить дружелюбное приветствие с помощью инструмента 'say_hello'. Больше ничего не делайте.",
        description="Обрабатывает простые приветствия, используя инструмент 'say_hello'.",
        tools=[say_hello],
    )
    print(f"✅ Агент '{greeting_agent.name}' переопределен.")
except Exception as e:
    print(f"❌ Не удалось переопределить Агента для приветствий. Ошибка: {e}")

# --- Переопределение Агента для прощаний (из Шага 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="Вы — Агент для прощаний. Ваша ЕДИНСТВЕННАЯ задача — предоставить вежливое прощальное сообщение с помощью инструмента 'say_goodbye'. Не выполняйте никаких других действий.",
        description="Обрабатывает простые прощания, используя инструмент 'say_goodbye'.",
        tools=[say_goodbye],
    )
    print(f"✅ Агент '{farewell_agent.name}' переопределен.")
except Exception as e:
    print(f"❌ Не удалось переопределить Агента для прощаний. Ошибка: {e}")

# --- Определение обновленного Корневого Агента ---
root_agent_stateful = None
runner_root_stateful = None # Инициализируем Runner

# Проверка наличия необходимых компонентов перед созданием корневого агента
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH # Выбираем модель для координации

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # Новое имя версии
        model=root_agent_model,
        description="Главный агент: Предоставляет погоду (с учетом единиц из состояния), делегирует приветствия/прощания, сохраняет отчет в состояние.",
        instruction="Вы — главный Агент Погоды. Ваша задача — предоставлять погоду с помощью инструмента 'get_weather_stateful'. "
                    "Инструмент отформатирует температуру в соответствии с предпочтениями пользователя, сохраненными в состоянии. "
                    "Делегируйте простые приветствия агенту 'greeting_agent', а прощания — агенту 'farewell_agent'. "
                    "Обрабатывайте только запросы о погоде, приветствия и прощания.",
        tools=[get_weather_stateful], # Используем инструмент, работающий с состоянием
        sub_agents=[greeting_agent, farewell_agent], # Подключаем субагентов
        output_key="last_weather_report" # <<< Автоматически сохранять итоговый ответ агента о погоде
    )
    print(f"✅ Корневой Агент '{root_agent_stateful.name}' создан с использованием stateful-инструмента и output_key.")

    # --- Создание Runner'а для этого Корневого Агента и НОВОГО Сервиса Сессий ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # Используем НОВЫЙ сервис сессий, работающий с состоянием
    )
    print(f"✅ Runner создан для stateful-корневого агента '{runner_root_stateful.agent.name}' с использованием stateful-сервиса сессий.")

else:
    print("❌ Невозможно создать stateful-корневого агента. Отсутствуют необходимые компоненты.")
    if not greeting_agent: print(" - Отсутствует определение greeting_agent.")
    if not farewell_agent: print(" - Отсутствует определение farewell_agent.")
    if 'get_weather_stateful' not in globals(): print(" - Отсутствует инструмент get_weather_stateful.")

✅ Агент 'greeting_agent' переопределен.
✅ Агент 'farewell_agent' переопределен.
✅ Корневой Агент 'weather_agent_v4_stateful' создан с использованием stateful-инструмента и output_key.
✅ Runner создан для stateful-корневого агента 'weather_agent_v4_stateful' с использованием stateful-сервиса сессий.


---

### 💬 4. Взаимодействуем и тестируем поток состояния

Теперь давайте выполним диалог, предназначенный для тестирования взаимодействий с состоянием, используя `runner_root_stateful` (связанный с нашим "сознательным" агентом и `session_service_stateful`). Мы будем использовать ранее определенную функцию `call_agent_async`, убедившись, что передаем правильный `runner`, ID пользователя (`USER_ID_STATEFUL`) и ID сессии (`SESSION_ID_STATEFUL`).

Поток диалога будет следующим:

1.  **Проверить погоду (Лондон):** Инструмент `get_weather_stateful` должен прочитать начальное предпочтение «Celsius» из состояния сессии, инициализированного в Разделе 1. Финальный ответ корневого агента (отчет о погоде в градусах Цельсия) должен быть сохранен в `state['last_weather_report']` через конфигурацию `output_key`.
2.  **Вручную обновить состояние:** Мы *напрямую изменим* состояние, хранящееся в экземпляре `InMemorySessionService`.
    > #### 📝 Примечание о прямом изменении состояния
    >
    > Метод `session_service.get_session()` возвращает *копию* сессии. Изменение этой копии не повлияет на состояние, используемое в последующих запусках агента. Для этого тестового сценария с `InMemorySessionService` мы получаем доступ к внутреннему словарю `sessions`, чтобы изменить *фактически хранящееся* значение состояния для `user_preference_temperature_unit` на «Fahrenheit». *В реальных приложениях изменения состояния обычно инициируются инструментами или логикой агента, возвращающими `EventActions(state_delta=...)`, а не прямыми ручными обновлениями.*
3.  **Снова проверить погоду (Нью-Йорк):** Инструмент `get_weather_stateful` теперь должен прочитать обновленное предпочтение «Fahrenheit» из состояния и соответствующим образом преобразовать температуру. *Новый* ответ корневого агента (погода в Фаренгейтах) перезапишет предыдущее значение в `state['last_weather_report']` из-за `output_key`.
4.  **Поприветствовать агента:** Проверяем, что делегирование `greeting_agent` по-прежнему работает корректно наряду с операциями с состоянием. Этот ответ станет *последним*, сохраненным `output_key` в этой последовательности.
5.  **Проверить финальное состояние:** После диалога мы извлекаем сессию в последний раз (получая копию) и выводим ее состояние, чтобы подтвердить, что `user_preference_temperature_unit` действительно «Fahrenheit», увидеть финальное значение, сохраненное `output_key` (в данном случае это будет приветствие), и значение `last_city_checked_stateful`, записанное инструментом.

In [21]:
# @title 4. Взаимодействие для теста потока состояний и output_key
import asyncio # Убедитесь, что asyncio импортирован

# Убедитесь, что stateful-runner (runner_root_stateful) доступен из предыдущей ячейки
# Убедитесь, что определены call_agent_async, USER_ID_STATEFUL, SESSION_ID_STATEFUL, APP_NAME

if 'runner_root_stateful' in globals() and runner_root_stateful:
    # Определяем основную асинхронную функцию для логики диалога с состоянием.
    # Ключевые слова 'await' ВНУТРИ этой функции необходимы для асинхронных операций.
    async def run_stateful_conversation():
        print("\n--- Тестирование состояния: Конвертация единиц температуры и output_key ---")

        # 1. Проверяем погоду (используется начальное состояние: Цельсий)
        print("--- Шаг 1: Запрос погоды в Лондоне (ожидается Цельсий) ---")
        await call_agent_async(query= "Какая погода в Лондоне?",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 2. Ручное обновление предпочтений в состоянии на Фаренгейт - ПРЯМОЕ ИЗМЕНЕНИЕ ХРАНИЛИЩА
        print("\n--- Ручное обновление состояния: Установка единиц в Фаренгейты ---")
        try:
            # Прямой доступ к внутреннему хранилищу - ЭТО СПЕЦИФИЧНО для InMemorySessionService в целях тестирования
            # ПРИМЕЧАНИЕ: В продакшене с постоянными сервисами (База данных, VertexAI) вы бы,
            # как правило, обновляли состояние через действия агента или специальные API сервиса,
            # а не путем прямого манипулирования внутренним хранилищем.
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Фаренгейт"
            # Опционально: вы можете также обновить временную метку, если какая-либо логика зависит от нее
            # import time
            # stored_session.last_update_time = time.time()
            print(f"--- Состояние сессии в хранилище обновлено. Текущее 'user_preference_temperature_unit': {stored_session.state.get('user_preference_temperature_unit', 'Не установлено')} ---")
        except KeyError:
            print(f"--- Ошибка: Не удалось получить сессию '{SESSION_ID_STATEFUL}' из внутреннего хранилища для пользователя '{USER_ID_STATEFUL}' в приложении '{APP_NAME}' для обновления состояния. Проверьте ID и была ли создана сессия. ---")
        except Exception as e:
             print(f"--- Ошибка при обновлении внутреннего состояния сессии: {e} ---")

        # 3. Проверяем погоду снова (инструмент теперь должен использовать Фаренгейт)
        # Это также обновит 'last_weather_report' через output_key
        print("\n--- Шаг 2: Запрос погоды в Нью-Йорке (ожидается Фаренгейт) ---")
        await call_agent_async(query= "Расскажи мне о погоде в Нью-Йорке.",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 4. Тестируем базовое делегирование (должно по-прежнему работать)
        # Это снова обновит 'last_weather_report', перезаписав отчет о погоде в Нью-Йорке
        print("\n--- Шаг 3: Отправка приветствия ---")
        await call_agent_async(query= "Привет!",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

    # --- Выполнение асинхронной функции `run_stateful_conversation` ---
    # Выберите ОДИН из методов ниже в зависимости от вашей среды.

    # МЕТОД 1: Прямой await (По умолчанию для Notebooks/Async REPL)
    print("Попытка выполнения через 'await' (по умолчанию для ноутбуков)...")
    await run_stateful_conversation()

    # МЕТОД 2: asyncio.run (Для стандартных Python-скриптов [.py])
    # Раскомментируйте следующий блок для запуска как скрипт:
    """
    import asyncio
    if __name__ == "__main__":
        print("Выполнение через 'asyncio.run()' (для стандартных Python-скриптов)...")
        try:
            asyncio.run(run_stateful_conversation())
        except Exception as e:
            print(f"Произошла ошибка: {e}")
    """

    # --- Проверка итогового состояния сессии после диалога ---
    # Этот блок выполняется после завершения любого из методов выполнения.
    print("\n--- Проверка итогового состояния сессии ---")
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id= USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Используем .get() для безопасного доступа к потенциально отсутствующим ключам
        print(f"Итоговое предпочтение: {final_session.state.get('user_preference_temperature_unit', 'Не установлено')}")
        print(f"Итоговый отчет о погоде (из output_key): {final_session.state.get('last_weather_report', 'Не установлено')}")
        print(f"Итоговый проверенный город (из инструмента): {final_session.state.get('last_city_checked_stateful', 'Не установлено')}")
        # print(f"Полный словарь состояния: {final_session.state.as_dict()}") # Для детального просмотра
    else:
        print("\n❌ Ошибка: не удалось получить итоговое состояние сессии.")

else:
    print("\n⚠️ Пропускаем выполнение тестового диалога. Stateful-runner ('runner_root_stateful') недоступен.")

Попытка выполнения через 'await' (по умолчанию для ноутбуков)...

--- Тестирование состояния: Конвертация единиц температуры и output_key ---
--- Шаг 1: Запрос погоды в Лондоне (ожидается Цельсий) ---

>>> Запрос пользователя: Какая погода в Лондоне?
--- Инструмент: get_weather_stateful вызван для города Лондон ---
--- Инструмент: Чтение состояния 'user_preference_temperature_unit': Цельсий ---
--- Инструмент: Сгенерирован отчет в Цельсий. Результат: {'status': 'success', 'report': 'В городе Лондон облачно, температура 15°C.'} ---
--- Инструмент: Обновлено состояние 'last_city_checked_stateful': Лондон ---
<<< Ответ агента: В городе Лондон облачно, температура 15°C.


--- Ручное обновление состояния: Установка единиц в Фаренгейты ---
--- Состояние сессии в хранилище обновлено. Текущее 'user_preference_temperature_unit': Фаренгейт ---

--- Шаг 2: Запрос погоды в Нью-Йорке (ожидается Фаренгейт) ---

>>> Запрос пользователя: Расскажи мне о погоде в Нью-Йорке.
--- Инструмент: get_weather_s

---

Проанализировав ход диалога и итоговое состояние сессии, вы можете подтвердить:

*   **Чтение состояния:** Погодный инструмент (`get_weather_stateful`) правильно прочитал `user_preference_temperature_unit` из состояния, первоначально используя «Celsius» для Лондона.
*   **Обновление состояния:** Прямое изменение успешно сменило сохраненное предпочтение на «Fahrenheit».
*   **Чтение обновленного состояния:** Инструмент впоследствии прочитал «Fahrenheit» при запросе погоды в Нью-Йорке и выполнил преобразование.
*   **Запись состояния инструментом:** Инструмент успешно записал `last_city_checked_stateful` («New York») в состояние через `tool_context.state`.
*   **Делегирование:** Делегирование `greeting_agent` для «Hi!» работало корректно даже после изменений состояния.
*   **`output_key`:** `output_key="last_weather_report"` успешно сохранял *финальный* ответ корневого агента за *каждый ход*, где корневой агент был конечным ответчиком. В этой последовательности последним ответом было приветствие («Hello, there!»), поэтому оно перезаписало отчет о погоде в ключе состояния.
*   **Финальное состояние:** Итоговая проверка подтверждает, что предпочтение сохранилось как «Fahrenheit».

Вы успешно интегрировали состояние сессии для персонализации поведения агента с помощью `ToolContext`, вручную управляли состоянием для тестирования `InMemorySessionService` и наблюдали, как `output_key` предоставляет простой механизм для сохранения последнего ответа агента. Это фундаментальное понимание управления состоянием является ключевым, поскольку на следующих шагах мы перейдем к реализации защитных механизмов с помощью колбэков.

## 🛡️ Шаг 5: Добавление безопасности — входной Guardrail с помощью `before_model_callback`

Наша команда агентов становится все более функциональной, запоминая предпочтения и эффективно используя инструменты. Однако в реальных сценариях нам часто требуются механизмы безопасности для контроля поведения агента *еще до того*, как потенциально проблемные запросы достигнут ядра большой языковой модели (LLM).

ADK предоставляет **колбэки (Callbacks)** — функции, которые позволяют вам встраиваться в определенные точки жизненного цикла выполнения агента. `before_model_callback` особенно полезен для обеспечения безопасности на входе.

### 🤔 Что такое `before_model_callback`?

*   Это определяемая вами Python-функция, которую ADK выполняет *непосредственно перед* тем, как агент отправит свой скомпилированный запрос (включая историю диалога, инструкции и последнее сообщение пользователя) базовой LLM.
*   **Цель:** Проверить запрос, при необходимости изменить его или полностью заблокировать на основе предопределенных правил.

#### Распространенные сценарии использования:

*   **Валидация/фильтрация ввода:** Проверка, соответствует ли ввод пользователя критериям или содержит ли он запрещенный контент (например, персональные данные или ключевые слова).
*   **Защитные механизмы (Guardrails):** Предотвращение обработки вредоносных, не относящихся к теме или нарушающих политику запросов со стороны LLM.
*   **Динамическое изменение промпта:** Добавление актуальной информации (например, из состояния сессии) в контекст запроса к LLM непосредственно перед отправкой.

> #### ⚙️ Как это работает:
>
> 1.  Вы определяете функцию, принимающую `callback_context: CallbackContext` и `llm_request: LlmRequest`.
>     *   `callback_context`: Предоставляет доступ к информации об агенте, состоянию сессии (`callback_context.state`) и т.д.
>     *   `llm_request`: Содержит полную полезную нагрузку, предназначенную для LLM (`contents`, `config`).
> 2.  Внутри функции:
>     *   **Проверить:** Изучить `llm_request.contents` (особенно последнее сообщение пользователя).
>     *   **Изменить (с осторожностью):** Вы *можете* изменять части `llm_request`.
>     *   **Заблокировать (Guardrail):** Вернуть объект `LlmResponse`. ADK немедленно отправит этот ответ обратно, *пропуская* вызов LLM на данном шаге.
>     *   **Разрешить:** Вернуть `None`. ADK продолжит выполнение и вызовет LLM с (возможно, измененным) запросом.

### ✅ На этом шаге мы:

1.  Определим функцию `before_model_callback` (`block_keyword_guardrail`), которая проверяет ввод пользователя на наличие определенного ключевого слова («BLOCK»).
2.  Обновим нашего "сознательного" корневого агента (`weather_agent_v4_stateful` из Шага 4), чтобы он использовал этот колбэк.
3.  Создадим новый `Runner`, связанный с этим обновленным агентом, но использующий *тот же сервис сессий с состоянием* для сохранения непрерывности состояния.
4.  Протестируем защитный механизм, отправляя как обычные запросы, так и запросы, содержащие ключевое слово.

---

### 🛡️ 1. Определяем функцию-колбэк для Guardrail

Эта функция будет проверять последнее сообщение пользователя в содержимом `llm_request`. Если она найдет «BLOCK» (без учета регистра), она создаст и вернет `LlmResponse` для блокировки потока; в противном случае она вернет `None`.

In [22]:
# @title 1. Определение защитного механизма (Guardrail) before_model_callback

# Убедитесь, что необходимые импорты доступны
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # Для создания контента ответа
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Description:
    ---------------
        Проверяет последнее сообщение пользователя на наличие слова 'BLOCK'. Если найдено,
        блокирует вызов LLM и возвращает предопределенный LlmResponse.
        В противном случае возвращает None, чтобы продолжить выполнение.

    Args:
    ---------------
        callback_context (CallbackContext): Контекст обратного вызова, предоставляющий доступ к имени агента и состоянию сессии.
        llm_request (LlmRequest): Объект запроса к LLM, содержащий историю диалога.

    Returns:
    ---------------
        Optional[LlmResponse]: Объект LlmResponse для блокировки вызова или None для его продолжения.
    """
    # Получаем имя агента, чей вызов модели перехватывается
    agent_name = callback_context.agent_name
    print(f"--- Callback: Защитный механизм block_keyword_guardrail запущен для агента: {agent_name} ---")

    # Извлекаем текст из последнего сообщения пользователя в истории запроса
    last_user_message_text = ""
    if llm_request.contents:
        # Находим самое последнее сообщение с ролью 'user'
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # Для простоты предполагаем, что текст находится в первой части
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # Нашли текст последнего сообщения пользователя

    # Логируем первые 100 символов для отладки
    print(f"--- Callback: Проверка последнего сообщения пользователя: '{last_user_message_text[:100]}...' ---")

    # --- Логика защитного механизма ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # Проверка без учета регистра
        print(f"--- Callback: Найдено '{keyword_to_block}'. Блокируем вызов LLM! ---")
        # Опционально, устанавливаем флаг в состоянии, чтобы зафиксировать событие блокировки
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- Callback: Установлено состояние 'guardrail_block_keyword_triggered': True ---")

        # Создаем и возвращаем LlmResponse, чтобы остановить поток и отправить этот ответ вместо реального
        return LlmResponse(
            content=types.Content(
                role="model", # Имитируем ответ с точки зрения агента
                parts=[types.Part(text=f"Я не могу обработать этот запрос, так как он содержит заблокированное ключевое слово '{keyword_to_block}'.")],
            )
            # Примечание: при необходимости здесь также можно установить поле error_message
        )
    else:
        # Ключевое слово не найдено, разрешаем продолжить запрос к LLM
        print(f"--- Callback: Ключевое слово не найдено. Разрешаем вызов LLM для {agent_name}. ---")
        return None # Возврат None сигнализирует ADK о необходимости продолжить работу в обычном режиме

print("✅ Функция защитного механизма block_keyword_guardrail определена.")

✅ Функция защитного механизма block_keyword_guardrail определена.


---

### 🌳 2. Обновляем корневого агента для использования колбэка

Мы переопределяем корневого агента, добавляя параметр `before_model_callback` и указывая на нашу новую функцию-guardrail. Для ясности мы дадим ему новое имя версии.

*Важно:* Нам нужно переопределить подагентов (`greeting_agent`, `farewell_agent`) и инструмент с состоянием (`get_weather_stateful`) в этом контексте, если они еще не доступны из предыдущих шагов, чтобы определение корневого агента имело доступ ко всем своим компонентам.

In [23]:
# @title 2. Обновление корневого агента с before_model_callback


# --- Переопределение субагентов (для гарантии их существования в этом контексте) ---
greeting_agent = None
try:
    # Используем определенную константу модели
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Сохраняем оригинальное имя для консистентности
        instruction="Вы — Агент для приветствий. Ваша ЕДИНСТВЕННАЯ задача — предоставить дружелюбное приветствие с помощью инструмента 'say_hello'. Больше ничего не делайте.",
        description="Обрабатывает простые приветствия, используя инструмент 'say_hello'.",
        tools=[say_hello],
    )
    print(f"✅ Субагент '{greeting_agent.name}' переопределен.")
except Exception as e:
    print(f"❌ Не удалось переопределить Агента для приветствий. Проверьте модель/API-ключ ({greeting_agent.model}). Ошибка: {e}")

farewell_agent = None
try:
    # Используем определенную константу модели
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Сохраняем оригинальное имя
        instruction="Вы — Агент для прощаний. Ваша ЕДИНСТВЕННАЯ задача — предоставить вежливое прощальное сообщение с помощью инструмента 'say_goodbye'. Не выполняйте никаких других действий.",
        description="Обрабатывает простые прощания, используя инструмент 'say_goodbye'.",
        tools=[say_goodbye],
    )
    print(f"✅ Субагент '{farewell_agent.name}' переопределен.")
except Exception as e:
    print(f"❌ Не удалось переопределить Агента для прощаний. Проверьте модель/API-ключ ({farewell_agent.model}). Ошибка: {e}")


# --- Определение Корневого Агента с Callback-функцией ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# Проверяем все компоненты перед продолжением
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # Используем определенную константу модели
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # Новое имя версии для ясности
        model=root_agent_model,
        description="Главный агент: Обрабатывает погоду, делегирует приветствия/прощания, включает защитный механизм для ключевых слов на входе.",
        instruction="Вы — главный Агент Погоды. Предоставляйте погоду с помощью 'get_weather_stateful'. "
                    "Делегируйте простые приветствия агенту 'greeting_agent', а прощания — 'farewell_agent'. "
                    "Обрабатывайте только запросы о погоде, приветствия и прощания.",
        tools=[get_weather_stateful], # ИСПРАВЛЕНО: Используем stateful-инструмент в соответствии с инструкцией
        sub_agents=[greeting_agent, farewell_agent], # Ссылаемся на переопределенных субагентов
        output_key="last_weather_report", # Сохраняем output_key из Шага 4
        before_model_callback=block_keyword_guardrail # <<< Назначаем callback-функцию защитного механизма
    )
    print(f"✅ Корневой Агент '{root_agent_model_guardrail.name}' создан с before_model_callback.")

    # --- Создание Runner'а для этого Агента с использованием ТОГО ЖЕ Stateful Сервиса Сессий ---
    # Убедимся, что session_service_stateful существует из Шага 4
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # Используем консистентное имя приложения
            session_service=session_service_stateful # <<< Используем сервис из Шага 4
        )
        print(f"✅ Runner создан для агента с защитным механизмом '{runner_root_model_guardrail.agent.name}', используя stateful-сервис сессий.")
    else:
        print("❌ Невозможно создать Runner. Отсутствует 'session_service_stateful' из Шага 4.")

else:
    print("❌ Невозможно создать корневой агент с защитным механизмом. Один или несколько необходимых компонентов отсутствуют или не были инициализированы:")
    if not greeting_agent: print("   - Агент для приветствий")
    if not farewell_agent: print("   - Агент для прощаний")
    if 'get_weather_stateful' not in globals(): print("   - инструмент 'get_weather_stateful'")
    if 'block_keyword_guardrail' not in globals(): print("   - callback-функция 'block_keyword_guardrail'")

✅ Субагент 'greeting_agent' переопределен.
✅ Субагент 'farewell_agent' переопределен.
✅ Корневой Агент 'weather_agent_v5_model_guardrail' создан с before_model_callback.
✅ Runner создан для агента с защитным механизмом 'weather_agent_v5_model_guardrail', используя stateful-сервис сессий.


---

### 💬 3. Взаимодействуем для тестирования Guardrail

Давайте протестируем поведение защитного механизма. Мы будем использовать *ту же сессию* (`SESSION_ID_STATEFUL`), что и на Шаге 4, чтобы показать, что состояние сохраняется при этих изменениях.

1.  Отправляем обычный запрос о погоде (должен пройти через guardrail и выполниться).
2.  Отправляем запрос, содержащий «BLOCK» (должен быть перехвачен колбэком).
3.  Отправляем приветствие (должно пройти через guardrail корневого агента, быть делегировано и выполниться нормально).

In [25]:
# @title 3. Взаимодействие для теста защитного механизма на входе модели
import asyncio # Убедитесь, что asyncio импортирован

# Убедитесь, что Runner для агента с защитным механизмом доступен
if 'runner_root_model_guardrail' in globals() and runner_root_model_guardrail:
    # Определяем основную асинхронную функцию для тестового диалога с защитным механизмом.
    # Ключевые слова 'await' ВНУТРИ этой функции необходимы для асинхронных операций.
    async def run_guardrail_test_conversation():
        print("\n--- Тестирование защитного механизма на входе модели ---")

        # Используем Runner для агента с callback-функцией и существующий ID stateful-сессии
        # Определяем вспомогательную лямбда-функцию для более чистых вызовов взаимодействия
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_model_guardrail,
                                                         USER_ID_STATEFUL, # Используем существующий ID пользователя
                                                         SESSION_ID_STATEFUL # Используем существующий ID сессии
                                                        )
        # 1. Обычный запрос (Ожидается, что Callback разрешит вызов, и будут использованы Фаренгейты из предыдущего состояния)
        print("--- Шаг 1: Запрос погоды в Лондоне (ожидается разрешение, Фаренгейт) ---")
        await interaction_func("Какая погода в Лондоне?")

        # 2. Запрос, содержащий заблокированное ключевое слово (Callback должен перехватить)
        print("\n--- Шаг 2: Запрос с заблокированным ключевым словом (ожидается блокировка) ---")
        await interaction_func("BLOCK запрос погоды в Токио") # Callback должен поймать "BLOCK"

        # 3. Обычное приветствие (Callback разрешает вызов корневому агенту, происходит делегирование)
        print("\n--- Шаг 3: Отправка приветствия (ожидается разрешение) ---")
        await interaction_func("Снова здравствуйте")

    # --- Выполнение асинхронной функции `run_guardrail_test_conversation` ---
    # Выберите ОДИН из методов ниже в зависимости от вашей среды.

    # МЕТОД 1: Прямой await (По умолчанию для Notebooks/Async REPL)
    print("Попытка выполнения через 'await' (по умолчанию для ноутбуков)...")
    await run_guardrail_test_conversation()

    # МЕТОД 2: asyncio.run (Для стандартных Python-скриптов [.py])
    # Раскомментируйте следующий блок для запуска как скрипт:
    """
    import asyncio
    if __name__ == "__main__":
        print("Выполнение через 'asyncio.run()' (для стандартных Python-скриптов)...")
        try:
            asyncio.run(run_guardrail_test_conversation())
        except Exception as e:
            print(f"Произошла ошибка: {e}")
    """

    # --- Проверка итогового состояния сессии после диалога ---
    # Этот блок выполняется после завершения любого из методов выполнения.
    # Опционально: Проверяем в состоянии флаг, установленный callback-функцией
    print("\n--- Проверка итогового состояния сессии (после теста защитного механизма) ---")
    # Используем экземпляр сервиса сессий, связанный с этой stateful-сессией
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Используем .get() для безопасного доступа
        print(f"Флаг срабатывания защитного механизма: {final_session.state.get('guardrail_block_keyword_triggered', 'Не установлено (или False)')}")
        print(f"Последний отчет о погоде: {final_session.state.get('last_weather_report', 'Не установлено')}") # Должна быть погода в Лондоне
        print(f"Единицы измерения температуры: {final_session.state.get('user_preference_temperature_unit', 'Не установлено')}") # Должны быть Фаренгейты
        # print(f"Полный словарь состояния: {final_session.state.as_dict()}") # Для детального просмотра
    else:
        print("\n❌ Ошибка: не удалось получить итоговое состояние сессии.")

else:
    print("\n⚠️ Пропускаем тест защитного механизма. Runner ('runner_root_model_guardrail') недоступен.")

Попытка выполнения через 'await' (по умолчанию для ноутбуков)...

--- Тестирование защитного механизма на входе модели ---
--- Шаг 1: Запрос погоды в Лондоне (ожидается разрешение, Фаренгейт) ---

>>> Запрос пользователя: Какая погода в Лондоне?
--- Callback: Защитный механизм block_keyword_guardrail запущен для агента: weather_agent_v5_model_guardrail ---
--- Callback: Проверка последнего сообщения пользователя: 'For context:...' ---
--- Callback: Ключевое слово не найдено. Разрешаем вызов LLM для weather_agent_v5_model_guardrail. ---
--- Инструмент: get_weather_stateful вызван для города Лондон ---
--- Инструмент: Чтение состояния 'user_preference_temperature_unit': Фаренгейт ---
--- Инструмент: Сгенерирован отчет в Фаренгейт. Результат: {'status': 'success', 'report': 'В городе Лондон облачно, температура 59°F.'} ---
--- Инструмент: Обновлено состояние 'last_city_checked_stateful': Лондон ---
--- Callback: Защитный механизм block_keyword_guardrail запущен для агента: weather_agent_v

---

Наблюдайте за потоком выполнения:

1.  **Погода в Лондоне:** Колбэк запускается для `weather_agent_v5_model_guardrail`, проверяет сообщение, выводит «Keyword not found. Allowing LLM call.» и возвращает `None`. Агент продолжает работу, вызывает инструмент `get_weather_stateful` (который использует предпочтение «Fahrenheit» из измененного на Шаге 4 состояния) и возвращает погоду. Этот ответ обновляет `last_weather_report` через `output_key`.
2.  **Запрос с BLOCK:** Колбэк снова запускается для `weather_agent_v5_model_guardrail`, проверяет сообщение, находит «BLOCK», выводит «Blocking LLM call!», устанавливает флаг в состоянии и возвращает предопределенный `LlmResponse`. Базовая LLM агента *никогда не вызывается* на этом шаге. Пользователь видит блокирующее сообщение от колбэка.
3.  **Снова привет:** Колбэк запускается для `weather_agent_v5_model_guardrail`, разрешает запрос. Затем корневой агент делегирует задачу `greeting_agent`. *Примечание: `before_model_callback`, определенный на корневом агенте, НЕ применяется автоматически к подагентам.* `greeting_agent` продолжает работу в обычном режиме, вызывает свой инструмент `say_hello` и возвращает приветствие.

Вы успешно реализовали уровень безопасности на входе! `before_model_callback` предоставляет мощный механизм для применения правил и контроля поведения агента *до* выполнения дорогостоящих или потенциально рискованных вызовов LLM. Далее мы применим аналогичную концепцию для добавления защитных механизмов вокруг самого использования инструментов.

## 🛡️ Шаг 6: Добавление безопасности — Guardrail для аргументов инструмента (`before_tool_callback`)

На Шаге 5 мы добавили защитный механизм для проверки и потенциальной блокировки ввода пользователя *до того*, как он достигнет LLM. Теперь мы добавим еще один уровень контроля — *после* того, как LLM решил использовать инструмент, но *до того*, как этот инструмент фактически выполнится. Это полезно для валидации *аргументов*, которые LLM хочет передать инструменту.

ADK предоставляет для этой цели `before_tool_callback`.

### 🤔 Что такое `before_tool_callback`?

*   Это Python-функция, которая выполняется непосредственно *перед* запуском конкретной функции-инструмента, после того как LLM запросил ее использование и определил аргументы.
*   **Цель:** Валидировать аргументы инструмента, предотвращать выполнение инструмента на основе определенных входных данных, динамически изменять аргументы или применять политики использования ресурсов.

#### Распространенные сценарии использования:

*   **Валидация аргументов:** Проверка, являются ли аргументы, предоставленные LLM, действительными, находятся ли они в допустимых диапазонах или соответствуют ли ожидаемым форматам.
*   **Защита ресурсов:** Предотвращение вызова инструментов с входными данными, которые могут быть дорогостоящими, обращаться к ограниченным данным или вызывать нежелательные побочные эффекты (например, блокировка вызовов API для определенных параметров).
*   **Динамическое изменение аргументов:** Корректировка аргументов на основе состояния сессии или другой контекстной информации перед запуском инструмента.

> #### ⚙️ Как это работает:
>
> 1.  Вы определяете функцию, принимающую `tool: BaseTool`, `args: Dict[str, Any]` и `tool_context: ToolContext`.
>     *   `tool`: Объект инструмента, который будет вызван (можно проверить `tool.name`).
>     *   `args`: Словарь аргументов, который LLM сгенерировал для инструмента.
>     *   `tool_context`: Предоставляет доступ к состоянию сессии (`tool_context.state`), информации об агенте и т.д.
> 2.  Внутри функции:
>     *   **Проверить:** Изучить `tool.name` и словарь `args`.
>     *   **Изменить:** Изменить значения в словаре `args` *напрямую*. Если вы вернете `None`, инструмент запустится с этими измененными аргументами.
>     *   **Заблокировать/Переопределить (Guardrail):** Вернуть **словарь**. ADK рассматривает этот словарь как *результат* вызова инструмента, полностью *пропуская* выполнение исходной функции-инструмента. Словарь в идеале должен соответствовать ожидаемому формату возвращаемых данных инструмента, который он блокирует.
>     *   **Разрешить:** Вернуть `None`. ADK продолжит и выполнит фактическую функцию-инструмент с (возможно, измененными) аргументами.

### ✅ На этом шаге мы:

1.  Определим функцию `before_tool_callback` (`block_paris_tool_guardrail`), которая специально проверяет, вызывается ли инструмент `get_weather_stateful` с городом «Paris».
2.  Если «Paris» обнаружен, колбэк заблокирует инструмент и вернет пользовательский словарь с ошибкой.
3.  Обновим нашего корневого агента (`weather_agent_v6_tool_guardrail`), чтобы он включал *оба* колбэка: `before_model_callback` и новый `before_tool_callback`.
4.  Создадим новый `Runner` для этого агента, используя тот же сервис сессий с состоянием.
5.  Протестируем поток, запрашивая погоду для разрешенных городов и для заблокированного города («Paris»).

---

### 🛡️ 1. Определяем функцию-колбэк для Tool Guardrail

Эта функция нацелена на инструмент `get_weather_stateful`. Она проверяет аргумент `city`. Если это «Paris», она возвращает словарь с ошибкой, который выглядит как собственный ответ инструмента об ошибке. В противном случае она позволяет инструменту выполниться, возвращая `None`.

In [26]:
# @title 1. Определение защитного механизма (Guardrail) before_tool_callback

# Убедитесь, что необходимые импорты доступны
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # Для подсказок типов

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Description:
    ---------------
        Проверяет, вызывается ли инструмент 'get_weather_stateful' для города 'Париж'.
        Если да, блокирует выполнение инструмента и возвращает специальный словарь с ошибкой.
        В противном случае, позволяет вызову инструмента продолжиться, возвращая None.

    Args:
    ---------------
        tool (BaseTool): Инструмент, который был вызван.
        args (Dict[str, Any]): Словарь с аргументами, переданными инструменту.
        tool_context (ToolContext): Контекст инструмента, предоставляющий доступ к имени агента и состоянию сессии.

    Returns:
    ---------------
        Optional[Dict]: Словарь для блокировки вызова или None для его продолжения.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # Агент, пытающийся вызвать инструмент
    print(f"--- Callback: Защитный механизм block_paris_tool_guardrail запущен для инструмента '{tool_name}' в агенте '{agent_name}' ---")
    print(f"--- Callback: Проверка аргументов: {args} ---")

    # --- Логика защитного механизма ---
    target_tool_name = "get_weather_stateful" # Должно совпадать с именем функции, используемой FunctionTool
    blocked_city = "париж" # ИСПРАВЛЕНО: Используем русское название для соответствия локализованному инструменту

    # Проверяем, является ли это нужным инструментом и совпадает ли аргумент города с заблокированным
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # Безопасно получаем аргумент 'city'
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Обнаружен заблокированный город '{city_argument}'. Блокируем выполнение инструмента! ---")
            # Опционально обновляем состояние
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Установлено состояние 'guardrail_tool_block_triggered': True ---")

            # Возвращаем словарь, соответствующий ожидаемому формату вывода инструмента для ошибок.
            # Этот словарь становится результатом работы инструмента, пропуская его реальный запуск.
            return {
                "status": "error",
                "error_message": f"Ограничение политики: Проверка погоды для города '{city_argument.capitalize()}' в данный момент отключена защитным механизмом инструмента."
            }
        else:
             print(f"--- Callback: Город '{city_argument}' разрешен для инструмента '{tool_name}'. ---")
    else:
        print(f"--- Callback: Инструмент '{tool_name}' не является целевым. Разрешаем выполнение. ---")


    # Если проверки выше не вернули словарь, разрешаем выполнение инструмента
    print(f"--- Callback: Разрешаем инструменту '{tool_name}' продолжить выполнение. ---")
    return None # Возврат None позволяет запустить реальную функцию инструмента

print("✅ Функция защитного механизма block_paris_tool_guardrail определена.")

✅ Функция защитного механизма block_paris_tool_guardrail определена.


---

### 🌳 2. Обновляем корневого агента для использования обоих колбэков

Мы снова переопределяем корневого агента (`weather_agent_v6_tool_guardrail`), на этот раз добавляя параметр `before_tool_callback` наряду с `before_model_callback` из Шага 5.

*Примечание о самодостаточном выполнении:* Как и на Шаге 5, убедитесь, что все предварительные условия (подагенты, инструменты, `before_model_callback`) определены или доступны в контексте выполнения перед определением этого агента.

In [ ]:
# @title 2. Update Root Agent with BOTH Callbacks (Self-Contained)

# --- Ensure Prerequisites are Defined ---
# (Include or ensure execution of definitions for: Agent, LiteLlm, Runner, ToolContext,
#  MODEL constants, say_hello, say_goodbye, greeting_agent, farewell_agent,
#  get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({greeting_agent.model}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({farewell_agent.model}). Error: {e}")

# --- Define the Root Agent with Both Callbacks ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # New version name
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # Keep model guardrail
        before_tool_callback=block_paris_tool_guardrail # <<< Add tool guardrail
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

    # --- Create Runner, Using SAME Stateful Session Service ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Use the service from Step 4/5
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")



✅ Субагент 'greeting_agent' переопределен.
✅ Субагент 'farewell_agent' переопределен.
✅ Корневой Агент 'weather_agent_v6_tool_guardrail' создан с ОБЕИМИ callback-функциями.
✅ Runner создан для агента с защитным механизмом инструментов 'weather_agent_v6_tool_guardrail', используя stateful-сервис сессий.


---

### 💬 3. Взаимодействуем для тестирования Tool Guardrail

Давайте протестируем поток взаимодействия, снова используя ту же сессию с состоянием (`SESSION_ID_STATEFUL`) из предыдущих шагов.

1.  Запрашиваем погоду для «New York»: Проходит оба колбэка, инструмент выполняется (используя предпочтение Fahrenheit из состояния).
2.  Запрашиваем погоду для «Paris»: Проходит `before_model_callback`. LLM решает вызвать `get_weather_stateful(city='Paris')`. `before_tool_callback` перехватывает вызов, блокирует инструмент и возвращает словарь с ошибкой. Агент передает эту ошибку.
3.  Запрашиваем погоду для «London»: Проходит оба колбэка, инструмент выполняется нормально.

In [30]:
# @title 3. Взаимодействие для теста защитного механизма аргументов инструмента
import asyncio # Убедитесь, что asyncio импортирован

# Убедитесь, что Runner для агента с защитным механизмом инструментов доступен
if 'runner_root_tool_guardrail' in globals() and runner_root_tool_guardrail:
    # Определяем основную асинхронную функцию для тестового диалога.
    # Ключевые слова 'await' ВНУТРИ этой функции необходимы для асинхронных операций.
    async def run_tool_guardrail_test():
        print("\n--- Тестирование защитного механизма аргументов инструмента ('Париж' заблокирован) ---")

        # Используем Runner для агента с обоими callback-ами и существующую stateful-сессию
        # Определяем вспомогательную лямбда-функцию для более чистых вызовов взаимодействия
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_tool_guardrail,
                                                         USER_ID_STATEFUL, # Используем существующий ID пользователя
                                                         SESSION_ID_STATEFUL # Используем существующий ID сессии
                                                        )
        # 1. Разрешенный город (Должен пройти оба callback-а, использовать состояние Фаренгейт)
        print("--- Шаг 1: Запрос погоды в Нью-Йорке (ожидается разрешение) ---")
        await interaction_func("Какая погода в Нью-Йорке?")

        # 2. Заблокированный город (Должен пройти callback модели, но быть заблокирован callback-ом инструмента)
        print("\n--- Шаг 2: Запрос погоды в Париже (ожидается блокировка защитным механизмом инструмента) ---")
        await interaction_func("А как насчет Парижа?") # Callback инструмента должен перехватить это

        # 3. Другой разрешенный город (Должен снова работать нормально)
        print("\n--- Шаг 3: Запрос погоды в Лондоне (ожидается разрешение) ---")
        await interaction_func("Расскажи мне о погоде в Лондоне.")

    # --- Выполнение асинхронной функции `run_tool_guardrail_test` ---
    # Выберите ОДИН из методов ниже в зависимости от вашей среды.

    # МЕТОД 1: Прямой await (По умолчанию для Notebooks/Async REPL)
    print("Попытка выполнения через 'await' (по умолчанию для ноутбуков)...")
    await run_tool_guardrail_test()

    # МЕТОД 2: asyncio.run (Для стандартных Python-скриптов [.py])
    # Раскомментируйте следующий блок для запуска как скрипт:
    """
    import asyncio
    if __name__ == "__main__":
        print("Выполнение через 'asyncio.run()' (для стандартных Python-скриптов)...")
        try:
            asyncio.run(run_tool_guardrail_test())
        except Exception as e:
            print(f"Произошла ошибка: {e}")
    """

    # --- Проверка итогового состояния сессии после диалога ---
    # Этот блок выполняется после завершения любого из методов выполнения.
    # Опционально: Проверяем в состоянии флаг срабатывания блокировки инструмента
    print("\n--- Проверка итогового состояния сессии (после теста защитного механизма инструмента) ---")
    # Используем экземпляр сервиса сессий, связанный с этой stateful-сессией
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id= SESSION_ID_STATEFUL)
    if final_session:
        # Используем .get() для безопасного доступа
        print(f"Флаг срабатывания защитного механизма инструмента: {final_session.state.get('guardrail_tool_block_triggered', 'Не установлено (или False)')}")
        print(f"Последний отчет о погоде: {final_session.state.get('last_weather_report', 'Не установлено')}") # Должна быть погода в Лондоне
        print(f"Единицы измерения температуры: {final_session.state.get('user_preference_temperature_unit', 'Не установлено')}") # Должны быть Фаренгейты
        # print(f"Полный словарь состояния: {final_session.state.as_dict()}") # Для детального просмотра
    else:
        print("\n❌ Ошибка: не удалось получить итоговое состояние сессии.")

else:
    print("\n⚠️ Пропускаем тест защитного механизма инструментов. Runner ('runner_root_tool_guardrail') недоступен.")

Попытка выполнения через 'await' (по умолчанию для ноутбуков)...

--- Тестирование защитного механизма аргументов инструмента ('Париж' заблокирован) ---
--- Шаг 1: Запрос погоды в Нью-Йорке (ожидается разрешение) ---

>>> Запрос пользователя: Какая погода в Нью-Йорке?
--- Callback: Защитный механизм block_keyword_guardrail запущен для агента: weather_agent_v6_tool_guardrail ---
--- Callback: Проверка последнего сообщения пользователя: 'For context:...' ---
--- Callback: Ключевое слово не найдено. Разрешаем вызов LLM для weather_agent_v6_tool_guardrail. ---
--- Callback: Защитный механизм block_paris_tool_guardrail запущен для инструмента 'get_weather_stateful' в агенте 'weather_agent_v6_tool_guardrail' ---
--- Callback: Проверка аргументов: {'city': 'Нью-Йорк'} ---
--- Callback: Город 'Нью-Йорк' разрешен для инструмента 'get_weather_stateful'. ---
--- Callback: Разрешаем инструменту 'get_weather_stateful' продолжить выполнение. ---
--- Инструмент: get_weather_stateful вызван для города

---

Проанализируйте вывод:

1.  **Нью-Йорк:** `before_model_callback` разрешает запрос. LLM запрашивает `get_weather_stateful`. `before_tool_callback` запускается, проверяет аргументы (`{'city': 'New York'}`), видит, что это не «Paris», выводит «Allowing tool...» и возвращает `None`. Фактическая функция `get_weather_stateful` выполняется, считывает «Fahrenheit» из состояния и возвращает отчет о погоде. Агент передает его, и он сохраняется через `output_key`.
2.  **Париж:** `before_model_callback` разрешает запрос. LLM запрашивает `get_weather_stateful(city='Paris')`. `before_tool_callback` запускается, проверяет аргументы, обнаруживает «Paris», выводит «Blocking tool execution!», устанавливает флаг в состоянии и возвращает словарь с ошибкой `{'status': 'error', 'error_message': 'Policy restriction...'}`. Фактическая функция `get_weather_stateful` **никогда не выполняется**. Агент получает словарь с ошибкой, *как если бы это был вывод инструмента*, и формулирует ответ на основе этого сообщения об ошибке.
3.  **Лондон:** Ведет себя так же, как Нью-Йорк, проходя оба колбэка и успешно выполняя инструмент. Новый отчет о погоде в Лондоне перезаписывает `last_weather_report` в состоянии.

Вы только что добавили критически важный уровень безопасности, контролирующий не только *что* достигает LLM, но и *как* могут использоваться инструменты агента на основе конкретных аргументов, сгенерированных LLM. Колбэки, такие как `before_model_callback` и `before_tool_callback`, необходимы для создания надежных, безопасных и соответствующих политикам агентных приложений.

---

## 🎉 Заключение: Ваша команда агентов готова!

Поздравляем! Вы успешно прошли путь от создания простого погодного агента до построения сложной многоагентной команды с помощью Agent Development Kit (ADK).

### 🚀 Давайте подытожим, чего вы достигли:

*   Вы начали с **фундаментального агента**, оснащенного одним инструментом (`get_weather`).
*   Вы изучили **мультимодельную гибкость** ADK с помощью LiteLLM, запуская одну и ту же базовую логику с различными LLM, такими как Gemini, GPT-4o и Claude.
*   Вы применили **модульность**, создав специализированных подагентов (`greeting_agent`, `farewell_agent`) и включив **автоматическое делегирование** от корневого агента.
*   Вы наделили своих агентов **памятью** с помощью **состояния сессии**, что позволило им запоминать предпочтения пользователя (`temperature_unit`) и прошлые взаимодействия (`output_key`).
*   Вы реализовали важнейшие **защитные механизмы (guardrails)**, используя как `before_model_callback` (блокировка определенных ключевых слов на входе), так и `before_tool_callback` (блокировка выполнения инструмента на основе аргументов, таких как город «Paris»).

Создавая эту прогрессивную команду погодных ботов, вы получили практический опыт работы с основными концепциями ADK, необходимыми для разработки сложных интеллектуальных приложений.

### 💡 Ключевые выводы:

*   **Агенты и инструменты:** Фундаментальные строительные блоки для определения возможностей и рассуждений. Четкие инструкции и докстринги имеют первостепенное значение.
*   **Runners и Session Services:** Движок и система управления памятью, которые оркестрируют выполнение агента и поддерживают контекст диалога.
*   **Делегирование:** Проектирование многоагентных команд обеспечивает специализацию, модульность и лучшее управление сложными задачами. `description` агента является ключом к автоматическому потоку.
*   **Состояние сессии (`ToolContext`, `output_key`):** Необходимо для создания контекстно-зависимых, персонализированных и многоходовых диалоговых агентов.
*   **Колбэки (`before_model`, `before_tool`):** Мощные хуки для реализации безопасности, валидации, применения политик и динамических изменений *перед* критически важными операциями (вызовами LLM или выполнением инструментов).
*   **Гибкость (`LiteLlm`):** ADK дает вам возможность выбирать лучшую LLM для работы, балансируя между производительностью, стоимостью и функциональностью.

### 🧭 Куда двигаться дальше?

Ваша команда погодных ботов — отличная отправная точка. Вот несколько идей для дальнейшего изучения ADK и улучшения вашего приложения:

1.  **Настоящий API погоды:** Замените `mock_weather_db` в вашем инструменте `get_weather` на вызов реального погодного API (например, OpenWeatherMap, WeatherAPI).
2.  **Более сложное состояние:** Храните больше предпочтений пользователя (например, предпочитаемое местоположение, настройки уведомлений) или сводки разговоров в состоянии сессии.
3.  **Уточнение делегирования:** Поэкспериментируйте с различными инструкциями для корневого агента или описаниями подагентов, чтобы точно настроить логику делегирования. Можете ли вы добавить агента для прогнозов?
4.  **Продвинутые колбэки:**
    *   Используйте `after_model_callback`, чтобы потенциально переформатировать или очистить ответ LLM *после* его генерации.
    *   Используйте `after_tool_callback` для обработки или логирования результатов, возвращаемых инструментом.
    *   Реализуйте `before_agent_callback` или `after_agent_callback` для логики входа/выхода на уровне агента.
5.  **Обработка ошибок:** Улучшите, как агент обрабатывает ошибки инструментов или неожиданные ответы API. Возможно, добавьте логику повторных попыток внутри инструмента.
6.  **Постоянное хранилище сессий:** Изучите альтернативы `InMemorySessionService` для постоянного хранения состояния сессии (например, с использованием баз данных, таких как Firestore или Cloud SQL — требует пользовательской реализации или будущих интеграций ADK).
7.  **Стриминговый UI:** Интегрируйте вашу команду агентов с веб-фреймворком (например, FastAPI, как показано в ADK Streaming Quickstart), чтобы создать чат-интерфейс в реальном времени.

Agent Development Kit предоставляет надежную основу для создания сложных приложений на базе LLM. Овладев концепциями, рассмотренными в этом руководстве — инструментами, состоянием, делегированием и колбэками, — вы хорошо подготовлены к решению все более сложных агентных систем.